# Read WMS Receipt

In [1]:
## run 1
import pandas as pd
import numpy as np
import pypyodbc

conn = pypyodbc.connect('Driver={SQL Server};'
                        'Server=NFI-DB-01;'
                        'Database=gudang;'
                        # 'UID=andra.miftah;'
                        # 'PWD=24nutrifoodindonesia21;'
                        'Trusted_Connection=yes;'
                )

cursor = conn.cursor()

wms_telkom = pd.read_sql(r"""SELECT t1.*, t2.*, t3.id, t3.sku, t3.name
    FROM [gudang].[dbo].[order_products] as t1
    left join orders as t2
    on t1.order_id = t2.id
    left join products as t3
    on t1.product_id = t3.id""", conn)
    
# wms_telkom

In [2]:
### for display only
pd.options.display.max_columns = 150

In [3]:
### tentative if need backup
wms_telkom.to_excel(r"Settlement Project\Manual\backup wms/data_wms_telkom 26 May 2023.xlsx", index = False)

In [1]:
### run 1 alternative
import pandas as pd
import numpy as np
import pypyodbc
# wms_telkom=pd.read_csv(r"C:\Users\timotius.giovandi\Documents\wmssep25.csv")
wms_telkom=pd.read_excel(r"Settlement Project\Manual\backup wms/data_wms_telkom Mar 2023.xlsx")

In [4]:
wms_telkom['order_date'].min()

Timestamp('1922-05-28 00:00:00')

In [5]:
### for backup
wms_telkom_cp = wms_telkom.copy()

In [6]:
wms_telkom['channel'].unique()

array(['Bukalapak', 'JD Indonesia', 'Lazada', 'Tokopedia', 'Nutrimart',
       'TikTok Commerce', 'Komplain', 'PPP', 'IndoHp', 'Sehatq',
       'Peminjaman', 'Nutrimart-WA', 'Grab Mart', 'Relationship',
       'Non Sales Order', 'nutrimart', 'Shopee', 'Aladin Mall', 'Orami',
       None, 'Evermos', 'NutriMart', 'Nonsales Order',
       '2022-06-20 06:03:15', '2022-06-19 23:25:12', 'PBG', 'Tik Tok',
       'Nutrmart'], dtype=object)

In [4]:
### for backup
wms_telkom = wms_telkom_cp.copy()

In [6]:
## run 2
## tambahan timo nov 2022 tarikan IT ada yang 1922 year nya

print(len(wms_telkom[wms_telkom['order_date'].astype(str).str.contains('1922')]))
wms_telkom.loc[wms_telkom['order_date'].astype(str).str.contains('1922'),'order_date']=wms_telkom['order_date'].astype(str).str.replace('1922','2022')
print(len(wms_telkom[wms_telkom['order_date'].astype(str).str.contains('1922')]))
print('harus null')

10
0
harus null


In [7]:
## run 3
wms_telkom['sku']=wms_telkom['sku'].astype(str) ### tambahan timo nov 2022 import dari excel tarikan IT

In [8]:
## run 4
wms_telkom_filt = wms_telkom[wms_telkom['channel'].isin(['IndoHp', 'Sehatq','Nutrimart-WA', 'Grab Mart', 'Orami','Evermos'])]

In [9]:
## run 5
wms_telkom_filt['channel'].unique()

array(['IndoHp', 'Sehatq', 'Nutrimart-WA', 'Grab Mart', 'Orami',
       'Evermos'], dtype=object)

In [10]:
## run 6
j_conv = pd.read_excel("Settlement Project\Rumus Realisasi.xlsx", sheet_name = 'Paket Brand')
j_conv = j_conv.iloc[:,:5]
j_conv = j_conv.rename(columns = {'SKU' : 'Real SKU', 'SKU.1' : 'SKU Produk', 'Name' : 'Nama Produk'})
j_conv = j_conv[j_conv['Real SKU'].notnull()]
j_conv['SKU Produk'] = j_conv['SKU Produk'].astype(str).str.replace('.0', '',regex = False)

In [11]:
## run 7
wms_telkom_filt = wms_telkom[wms_telkom['channel'].isin(['IndoHp', 'Sehatq','Nutrimart-WA', 'Grab Mart', 'Orami','Evermos'])]

colname = wms_telkom_filt.columns.to_list() + ['Bundle Name']
wms_telkom_filt_all = wms_telkom_filt.merge(j_conv, how = 'left', left_on = ['sku'], right_on = ['Real SKU'])
wms_j = wms_telkom_filt_all[wms_telkom_filt_all['SKU Produk'].notnull()]

wms_j['sku'] = wms_j['SKU Produk']
wms_j['Bundle Name'] = wms_j['name']
wms_j['name'] = wms_j['Nama Produk']
wms_j['quantity'] = wms_j['quantity'] * wms_j['Konversi ke UOM'].astype(float)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [12]:
## run 8
dropped = [x for x in wms_j.columns if x not in wms_telkom_filt.columns]
wms_j = wms_j.drop(dropped, axis = 1)

In [13]:
## run 9
wms_telkom_filt = wms_telkom[wms_telkom['channel'].isin(['IndoHp', 'Sehatq','Nutrimart-WA', 'Grab Mart', 'Orami','Evermos'])]

dropped = [x for x in wms_telkom_filt.columns if wms_telkom_filt.columns.to_list().count(x) > 1]
wms_telkom_filt = wms_telkom_filt.drop(dropped, axis = 1)

colname = wms_telkom_filt.columns.to_list() + ['Bundle Name']
wms_telkom_filt_all = wms_telkom_filt.merge(j_conv, how = 'left', left_on = ['sku'], right_on = ['Real SKU'])
wms_j = wms_telkom_filt_all[wms_telkom_filt_all['SKU Produk'].notnull()]

wms_j['sku'] = wms_j['SKU Produk']
wms_j['Bundle Name'] = wms_j['name']
wms_j['name'] = wms_j['Nama Produk']
wms_j['quantity'] = wms_j['quantity'] * wms_j['Konversi ke UOM'].astype(float)

# dropped = [x for x in wms_j.columns if x not in wms_telkom_filt.columns and x != 'Bundle Name']
# wms_j = wms_j.drop(dropped, axis = 1)

wms_telkom_filt = pd.concat([wms_telkom_filt, wms_j])
wms_telkom_filt = wms_telkom_filt[colname]

wms_telkom_filt['Paid Date'] = pd.to_datetime(wms_telkom_filt['order_date'])
wms_telkom_filt['Order Date'] = pd.to_datetime(wms_telkom_filt['order_date'])
wms_telkom_filt['True datetime'] = pd.to_datetime(wms_telkom_filt['order_date'])
wms_telkom_filt['Bundle Flag'] = np.nan
wms_telkom_filt['Store Type'] = np.nan
wms_telkom_filt['Store'] = wms_telkom_filt['channel']

wms_telkom_filt = wms_telkom_filt.rename(columns = {
    'order_number' : 'Order #',
     'invoice_number' : 'Sales Order ID',
    'awb_number' : 'AWB',
    'channel' : 'Channel',
    'sku' : 'SKU',
    'name' : 'Product Name',
    'quantity' : 'Qty. Invoiced',
    'shipping_method' : 'Shipping Courier',
    'shipping_province' : 'Region Group',
    'status' : 'Order Status'    
})


wms_telkom_filt['Week'] = wms_telkom_filt['Order Date'].dt.week
wms_telkom_filt['Date'] = wms_telkom_filt['Order Date'].dt.day
wms_telkom_filt['Month'] = wms_telkom_filt['Order Date'].dt.month_name()
wms_telkom_filt['Year'] = wms_telkom_filt['Order Date'].dt.year

wms_telkom_filt['Warehouse Quantity'] = wms_telkom_filt['Qty. Invoiced']

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithC

In [14]:
## run 10
indeks = wms_telkom_filt[(pd.to_datetime(wms_telkom_filt['Order Date']).dt.strftime('%d').astype(int) <= 12) & (wms_telkom_filt['Channel'] == 'Sehatq')].index.to_list()
wms_telkom_filt['Date'][indeks] = pd.to_datetime(pd.to_datetime(wms_telkom_filt['Order Date'][indeks]).dt.strftime('%Y-%d-%m %H:%M')).dt.day
wms_telkom_filt['Month'][indeks] = pd.to_datetime(pd.to_datetime(wms_telkom_filt['Order Date'][indeks]).dt.strftime('%Y-%d-%m %H:%M')).dt.month_name()
wms_telkom_filt['Year'][indeks] = pd.to_datetime(pd.to_datetime(wms_telkom_filt['Order Date'][indeks]).dt.strftime('%Y-%d-%m %H:%M')).dt.year

non_indeks = wms_telkom_filt[~wms_telkom_filt.index.isin(indeks) & (wms_telkom_filt['Channel'] == 'Sehatq')].index.to_list()
wms_telkom_filt['Date'][non_indeks] = pd.to_datetime(wms_telkom_filt['Order Date'][non_indeks]).dt.day
wms_telkom_filt['Month'][non_indeks] = pd.to_datetime(wms_telkom_filt['Order Date'][non_indeks]).dt.month_name()
wms_telkom_filt['Year'][non_indeks] = pd.to_datetime(wms_telkom_filt['Order Date'][non_indeks]).dt.year

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [15]:
## run 11
wms_telkom_filt = wms_telkom_filt[[x for x in wms_telkom_filt.columns if x in ['Order #',
 'Sales Order ID',
 'AWB',
 'Paid Date',
 'Order date',
 'Week',
 'Date',
 'Month',
 'Quarter',
 'Year',
 'Channel',
 'SKU',
 'Brand',
 'Product Name',
 'Bundle Name',
 'Price List NFI',
 'Qty. Invoiced',
 'Total Net',
 'Sub Brand',
 'Real SKU',
 'Real Nama Produk',
 'Parent Item',
 'Parent SKU',
 'Bundle Flag',
 'Shipping Courier',
 'Store',
 'True datetime',
 'Store Type',
 'Region Group',
 'PL Before PPN',
 'Total Net Before PPN',
 'Category Baru',
 'Exported Parent Item',
 'Order Date Seller Centre',
 'Product ID',
 'Total Selcen',
 'Order Status',
 'Warehouse Origin',
 'Paid',
 'SO STATUS',
 'WH',
 'Warehouse Quantity',
 'Total Produk Prop',
 'Fullfee Prop',
 'Comfee Prop',
 'BTL Cost',
 'Total Produk Prop Adj',     
 ]]]

In [16]:
## run 12
temp2 = wms_telkom_filt.copy()

import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os
    
data_SKU = pd.read_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx")

s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

to_excel = data_SKU.to_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx", index = False)
data_SKU.loc[data_SKU['Price List NFI'].isin(['-']), 'Price List NFI'] = 0
data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')

data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

list_col = ['Real SKU', 'Real Nama Produk']

temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
for i in list_col:
    temp2[i] = temp_merge[str(i)]

list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(temp2.index)
list_pcs = [x for x in temp_merge.columns if 'PCS' in x and '_y' in x]
for i in list_pcs:
    temp_merge[i] = temp_merge[i] * temp_merge['Qty. Invoiced']

list_col = data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()    
for i in list_col:
    temp2[i] = temp_merge[str(i)]
    
    
print("Unbundling")
data_bundle1 = temp2[~temp2['Produk 1'].isnull()]
data_bundle1['Bundle Name'] = data_bundle1['Product Name']
data_bundle1['Product Name'] = data_bundle1['Produk 1']
data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
data_bundle1['Bundle Flag'] = np.nan

data_bundle2 = temp2[~temp2['Produk 2'].isnull()]
data_bundle2['Bundle Name'] = data_bundle2['Product Name']
data_bundle2['Product Name'] = data_bundle2['Produk 2']
data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
data_bundle2['Bundle Flag'] = np.nan

data_bundle3 = temp2[~temp2['Produk 3'].isnull()]
data_bundle3['Bundle Name'] = data_bundle3['Product Name']
data_bundle3['Product Name'] = data_bundle3['Produk 3']
data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
data_bundle3['Bundle Flag'] = np.nan

data_bundle4 = temp2[~temp2['Produk 4'].isnull()]
data_bundle4['Bundle Name'] = data_bundle4['Product Name']
data_bundle4['Product Name'] = data_bundle4['Produk 4']
data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
data_bundle4['Bundle Flag'] = np.nan

data_bundle5 = temp2[~temp2['Produk 5'].isnull()]
data_bundle5['Bundle Name'] = data_bundle5['Product Name']
data_bundle5['Product Name'] = data_bundle5['Produk 5']
data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
data_bundle5['Bundle Flag'] = np.nan

data_bundle6 = temp2[~temp2['Produk 6'].isnull()]
data_bundle6['Bundle Name'] = data_bundle6['Product Name']
data_bundle6['Product Name'] = data_bundle6['Produk 6']
data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
data_bundle6['Bundle Flag'] = np.nan

data_bundle7 = temp2[~temp2['Produk 7'].isnull()]
data_bundle7['Bundle Name'] = data_bundle7['Product Name']
data_bundle7['Product Name'] = data_bundle7['Produk 7']
data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
data_bundle7['Bundle Flag'] = np.nan

data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
data_bundle['SKU'] = data_bundle['SKU'].astype(str)
data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']]

print("Pricing")
temp2 = temp2.append(data_bundle, ignore_index = True, sort = False)
list_col = ['Real SKU', 'Price List NFI', 'Harga Cost']

temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
for i in list_col:
    if i == 'Real SKU' or i == 'Price List NFI':
        temp2[i] = temp_merge[str(i) + '_y']
    else :
        temp2[i] = temp_merge[str(i)]
    
temp2['Price List NFI'] = pd.to_numeric(temp2['Price List NFI']).astype(int)
temp2['Harga Cost'] = pd.to_numeric(temp2['Harga Cost']).astype(int)
temp2['Qty. Invoiced'] = pd.to_numeric(temp2['Qty. Invoiced'])

temp2['Total Net'] = temp2['Price List NFI'] * temp2['Qty. Invoiced']
temp2['Total Harga Cost'] = temp2['Harga Cost'] * temp2['Qty. Invoiced']

indeks = temp2[temp2['Product Name'].isin(wms_j['Bundle Name'])].index.to_list()
temp2['Brand'][indeks] = 'Bundle'

Unbundling
Pricing


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [17]:
## run 13
temp2[temp2['Price List NFI'].isnull()]
## harus kosong yaaa

,Qty. Invoiced,Order Status,Order #,Shipping Courier,Sales Order ID,AWB,Channel,Region Group,SKU,Product Name,Bundle Name,Paid Date,True datetime,Bundle Flag,Store Type,Store,Week,Date,Month,Year,Warehouse Quantity,Real SKU,Real Nama Produk,Produk 1,SKU Produk 1,PCS Produk 1,Price List NFI 1,Subtotal Produk 1,Harga Display 1,Harga Cost 1,Harga Organik 1,Produk 2,SKU Produk 2,PCS Produk 2,Price List NFI 2,Subtotal Produk 2,Harga Display 2,Harga Cost 2,Harga Organik 2,Produk 3,SKU Produk 3,PCS Produk 3,Price List NFI 3,Subtotal Produk 3,Harga Display 3,Harga Cost 3,Harga Organik 3,Produk 4,SKU Produk 4,PCS Produk 4,Price List NFI 4,Subtotal Produk 4,Harga Display 4,Harga Cost 4,Harga Organik 4,Produk 5,SKU Produk 5,PCS Produk 5,Price List NFI 5,Subtotal Produk 5,Harga Display 5,Harga Cost 5,Harga Organik 5,Produk 6,SKU Produk 6,PCS Produk 6,Price List NFI 6,Subtotal Produk 6,Harga Display 6,Harga Cost 6,Harga Organik 6,Produk 7,SKU Produk 7,PCS Produk 7,Price List NFI 7,Subtotal Produk 7,Harga Display 7,Harga Cost 7,Harga Organik 7,Price List NFI,Total Net,Brand,Sub Brand,Parent Item,Parent SKU,Harga Cost,Total Harga Cost


In [18]:
## run 14
temp2['Total Net Before PPN'] = temp2['Total Net']/1.11

In [19]:
## run 15
wms_telkom_filt = temp2.copy()

In [20]:
## run 16
wms_telkom_filt['Channel'].unique()

array(['IndoHp', 'Sehatq', 'Nutrimart-WA', 'Grab Mart', 'Orami',
       'Evermos'], dtype=object)

In [21]:
## run 17
wms_telkom_filt[['Month', 'Year']].drop_duplicates()
# temp2
wms_telkom_filt[wms_telkom_filt['Year']==1922]

,Qty. Invoiced,Order Status,Order #,Shipping Courier,Sales Order ID,AWB,Channel,Region Group,SKU,Product Name,Bundle Name,Paid Date,True datetime,Bundle Flag,Store Type,Store,Week,Date,Month,Year,Warehouse Quantity,Real SKU,Real Nama Produk,Produk 1,SKU Produk 1,PCS Produk 1,Price List NFI 1,Subtotal Produk 1,Harga Display 1,Harga Cost 1,Harga Organik 1,Produk 2,SKU Produk 2,PCS Produk 2,Price List NFI 2,Subtotal Produk 2,Harga Display 2,Harga Cost 2,Harga Organik 2,Produk 3,SKU Produk 3,PCS Produk 3,Price List NFI 3,Subtotal Produk 3,Harga Display 3,Harga Cost 3,Harga Organik 3,Produk 4,SKU Produk 4,PCS Produk 4,Price List NFI 4,Subtotal Produk 4,Harga Display 4,Harga Cost 4,Harga Organik 4,Produk 5,SKU Produk 5,PCS Produk 5,Price List NFI 5,Subtotal Produk 5,Harga Display 5,Harga Cost 5,Harga Organik 5,Produk 6,SKU Produk 6,PCS Produk 6,Price List NFI 6,Subtotal Produk 6,Harga Display 6,Harga Cost 6,Harga Organik 6,Produk 7,SKU Produk 7,PCS Produk 7,Price List NFI 7,Subtotal Produk 7,Harga Display 7,Harga Cost 7,Harga Organik 7,Price List NFI,Total Net,Brand,Sub Brand,Parent Item,Parent SKU,Harga Cost,Total Harga Cost,Total Net Before PPN


In [23]:
wms_telkom_filt['Store'].unique()

array(['IndoHp', 'Sehatq', 'Nutrimart-WA', 'Grab Mart', 'Orami',
       'Evermos'], dtype=object)

In [24]:
## for backup
wms_telkom_filt_copy=wms_telkom_filt.copy()

In [82]:
## for backup
wms_telkom_filt=wms_telkom_filt_copy.copy()

# IndoHP

In [22]:
## run 18
indohp = pd.read_excel(r"Settlement Project\Manual\Settlement Indohp 2022 - ok.xlsx")

In [23]:
## run 19
for i in indohp.columns:
    if 'Unnamed' in i:
        indohp = indohp.drop(i, axis = 1)
indohp

wms_indohp = wms_telkom_filt[wms_telkom_filt['Store'] == 'IndoHp']
wms_indohp['Order #'] = wms_indohp['Order #'].astype(str).str.strip()
indohp['No Order'] = indohp['No Order'].astype(str).str.strip()
wms_indohp = wms_indohp.merge(indohp[['No Order', 'Nominal']], how = 'left', left_on = 'Order #', right_on = 'No Order')

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
wms_indohp.columns

Index(['Qty. Invoiced', 'Order Status', 'Order #', 'Shipping Courier',
       'Sales Order ID', 'AWB', 'Channel', 'Region Group', 'SKU',
       'Product Name', 'Bundle Name', 'Paid Date', 'True datetime',
       'Bundle Flag', 'Store Type', 'Store', 'Week', 'Date', 'Month', 'Year',
       'Warehouse Quantity', 'Real SKU', 'Real Nama Produk', 'Produk 1',
       'SKU Produk 1', 'PCS Produk 1', 'Price List NFI 1', 'Subtotal Produk 1',
       'Harga Display 1', 'Harga Cost 1', 'Harga Organik 1', 'Produk 2',
       'SKU Produk 2', 'PCS Produk 2', 'Price List NFI 2', 'Subtotal Produk 2',
       'Harga Display 2', 'Harga Cost 2', 'Harga Organik 2', 'Produk 3',
       'SKU Produk 3', 'PCS Produk 3', 'Price List NFI 3', 'Subtotal Produk 3',
       'Harga Display 3', 'Harga Cost 3', 'Harga Organik 3', 'Produk 4',
       'SKU Produk 4', 'PCS Produk 4', 'Price List NFI 4', 'Subtotal Produk 4',
       'Harga Display 4', 'Harga Cost 4', 'Harga Organik 4', 'Produk 5',
       'SKU Produk 5', 'PCS Prod

In [25]:
## run 20
non_bundle = wms_indohp[wms_indohp['Bundle Name'].isnull()]
bundling = wms_indohp[wms_indohp['Bundle Name'].notnull()]

non_bundle['Total Produk'] = non_bundle['Nominal']
non_bundle['Total Produk Prop'] = non_bundle['Total Produk'].fillna(0) * wms_indohp['Total Net Before PPN']/wms_indohp.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

bundling = bundling.merge(non_bundle[['Order #', 'Product Name', 'Total Produk Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])
bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')

wms_indohp = non_bundle.append(bundling, ignore_index = True, sort = False)

In [26]:
wms_indohp[wms_indohp['Bundle Name'].notnull()]

,Qty. Invoiced,Order Status,Order #,Shipping Courier,Sales Order ID,AWB,Channel,Region Group,SKU,Product Name,Bundle Name,Paid Date,True datetime,Bundle Flag,Store Type,Store,Week,Date,Month,Year,Warehouse Quantity,Real SKU,Real Nama Produk,Produk 1,SKU Produk 1,PCS Produk 1,Price List NFI 1,Subtotal Produk 1,Harga Display 1,Harga Cost 1,Harga Organik 1,Produk 2,SKU Produk 2,PCS Produk 2,Price List NFI 2,Subtotal Produk 2,Harga Display 2,Harga Cost 2,Harga Organik 2,Produk 3,SKU Produk 3,PCS Produk 3,Price List NFI 3,Subtotal Produk 3,Harga Display 3,Harga Cost 3,Harga Organik 3,Produk 4,SKU Produk 4,PCS Produk 4,Price List NFI 4,Subtotal Produk 4,Harga Display 4,Harga Cost 4,Harga Organik 4,Produk 5,SKU Produk 5,PCS Produk 5,Price List NFI 5,Subtotal Produk 5,Harga Display 5,Harga Cost 5,Harga Organik 5,Produk 6,SKU Produk 6,PCS Produk 6,Price List NFI 6,Subtotal Produk 6,Harga Display 6,Harga Cost 6,Harga Organik 6,Produk 7,SKU Produk 7,PCS Produk 7,Price List NFI 7,Subtotal Produk 7,Harga Display 7,Harga Cost 7,Harga Organik 7,Price List NFI,Total Net,Brand,Sub Brand,Parent Item,Parent SKU,Harga Cost,Total Harga Cost,Total Net Before PPN,No Order,Nominal,Total Produk,Total Produk Prop,Product Name_x,Product Name_y


In [27]:
## run 21
wms_indohp = wms_indohp.drop(['No Order', 'Nominal'], axis = 1)

In [28]:
## run 22
wms_telkom_filt = wms_telkom_filt[wms_telkom_filt['Store'] != 'IndoHp']
wms_telkom_filt = wms_telkom_filt.append(wms_indohp, ignore_index = True, sort = False)

# evermos

In [29]:
## run 17.1
evermos = pd.read_excel(r"D:\6. settlement\Code\Settlement Project\Manual\12. Settlement Evermos 2023.xlsx")

In [30]:
## run 17.2
for i in evermos.columns:
    if 'Unnamed' in i:
        evermos = evermos.drop(i, axis = 1)
evermos

wms_evermos = wms_telkom_filt[wms_telkom_filt['Store'] == 'Evermos']
wms_evermos['Order #'] = wms_evermos['Order #'].astype(str).str.strip()
evermos['No Order'] = evermos['ID Order'].astype(str).str.strip()
evermos['Nominal']=evermos['Amount']
wms_evermos = wms_evermos.merge(evermos[['No Order', 'Nominal']], how = 'left', left_on = 'Order #', right_on = 'No Order')

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
## run 17.3

if 'Product Name_y' in wms_evermos.columns:
    wms_evermos = wms_evermos.drop('Product Name_y', axis = 1)#.rename(columns = {'Product Name_x' : 'Product Name'})
if 'Product Name_x' in wms_evermos.columns:
    wms_evermos = wms_evermos.drop('Product Name_x', axis = 1)

non_bundle = wms_evermos[wms_evermos['Bundle Name'].isnull()]
bundling = wms_evermos[wms_evermos['Bundle Name'].notnull()]

non_bundle['Total Produk'] = non_bundle['Nominal']
non_bundle['Total Produk Prop'] = non_bundle['Total Produk'].fillna(0) * wms_evermos['Total Net Before PPN']/wms_evermos.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

bundling = bundling.drop(['Total Produk Prop'], axis = 1)
bundling = bundling.merge(non_bundle[['Order #', 'Product Name', 'Total Produk Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])
bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')

# non_bundle = non_bundle.drop('Product Name_y', axis = 1).rename(columns = {'Product Name_x' : 'Product Name'})
# bundling = bundling.drop('Product Name_y', axis = 1).rename(columns = {'Product Name_x' : 'Product Name'})
wms_evermos = non_bundle.append(bundling, ignore_index = True, sort = False)

In [32]:
## run 17.4
wms_evermos = wms_evermos.drop(['No Order', 'Nominal'], axis = 1)

In [33]:
## run 17.5
wms_telkom_filt = wms_telkom_filt[wms_telkom_filt['Store'] != 'Evermos']
wms_telkom_filt = wms_telkom_filt.append(wms_evermos, ignore_index = True, sort = False)

# Sehatq

In [34]:
## run 23
sehatq22 = pd.read_excel(r"Settlement Project\Manual\Settlement SehatQ 2022-ok.xlsx")

sehatq22 = sehatq22[sehatq22['Transaction id'].notnull()]
sehatq22 = sehatq22[sehatq22['Transaction id'] != 'Mo Number']
for i in sehatq22.columns:
    if 'Unnamed' in i:
        sehatq22 = sehatq22.drop(i, axis = 1)
sehatq22


sehatq23 = pd.read_excel(r"Settlement Project\Manual\10. Settlement SehatQ 2023.xlsx")
sehatq23=sehatq23.rename(columns={'Mo Number':'Transaction id','Paid Date':'Transaction date','Complete Date':'Complete date','Name Product':'Product Name','qty':'Qty','Total Harga Product':'Selling Price','PPN 11%':'PPN 10%','Transaction Value':'Transaction value'})[sehatq22.columns]
sehatq=pd.concat([sehatq22,sehatq23],ignore_index=True)

In [35]:
## run 24
sehatq = sehatq[sehatq['Transaction id'].notnull()]
sehatq = sehatq[sehatq['Transaction id'] != 'Mo Number']
for i in sehatq.columns:
    if 'Unnamed' in i:
        sehatq = sehatq.drop(i, axis = 1)
sehatq

,No,Transaction id,Transaction date,Complete date,Product Name,Qty,Selling Price,Fee,Fee Rp,PPN 10%,Transaction value
0,1,TQ0921R9RIQ5-01,2021-09-28 00:00:00,2021-09-30 00:00:00,HiLo Gold Chocolate 750 g,1,105000,0.05,5250,525,99225
1,2,TQ0921SXLHH5-01,2021-09-29 00:00:00,2021-09-29 00:00:00,HiLo Active Ketan Hitam,1,30000,0.05,1500,150,28350
2,3,TQ0921SXLHH5-01,2021-09-29 00:00:00,2021-09-29 00:00:00,L-Men Protein Crunch BBQ Beef 20 g,2,20000,0.05,1000,100,18900
3,4,TQ0921SXLHH5-01,2021-09-29 00:00:00,2021-09-29 00:00:00,Tropicana Slim Avocado Coffee (1 Box @ 4 Sachet),1,6360,0.05,318,31.8,6010.2
4,1,TQ1121LPC0QL-01,2021-11-20 00:00:00,2021-11-22 00:00:00,Tropicana Slim Mayonnaise 200 g,1,20200,0.05,1010,101,19089
...,...,...,...,...,...,...,...,...,...,...,...
82,1,TQ0223OUAXTS-02,2/28/2023 19:38,3/7/2023 13:47,Tropicana Slim Sweetener I-Sweet (1 Box @ 25 S...,2,58000,0.05,2900,319,54781
83,2,TQ0223OUAXTS-02,2/28/2023 19:38,3/7/2023 13:47,Tropicana Slim Klepon Cookies (1 Box @ 5 Sachet),3,80400,0.05,4020,442.2,75937.8
84,3,TQ0223OUAXTS-02,2/28/2023 19:38,3/7/2023 13:47,Tropicana Slim Korean Goguma Cookies (1 Box @ ...,3,80400,0.05,4020,442.2,75937.8
85,4,TQ0223OUAXTS-02,2/28/2023 19:38,3/7/2023 13:47,Tropicana Slim Sweet Orange (1 Box @ 10 Sachet),2,46200,0.05,2310,254.1,43635.9


In [36]:
## run 25
sehatq_sku = pd.read_excel(r'Settlement Project/Manual/active sehatq 6 january 2022.xlsx')

sehatq['Product Name'] = sehatq['Product Name'].str.strip()
sehatq['Transaction id'] = sehatq['Transaction id'].str.strip()
sehatq_sku['Nama Produk'] = sehatq_sku['Nama Produk'].str.strip()

sehatq = sehatq.merge(sehatq_sku[['Nama Produk', 'Sku Merchant']].drop_duplicates('Nama Produk'), how = 'left', left_on = 'Product Name', right_on = 'Nama Produk')

In [37]:
## run 26
wms_sehatq = wms_telkom_filt[wms_telkom_filt['Store'] == 'Sehatq']
wms_sehatq = wms_sehatq.drop(['Product Name_x', 'Product Name_y'], axis = 1) ##### ini timo comment 25 may


sehatq['Transaction id'] = sehatq['Transaction id'].astype(str)
wms_sehatq['Order #'] = wms_sehatq['Order #'].astype(str)
wms_sehatq['Real SKU'] = wms_sehatq['Real SKU'].astype(str)
sehatq['Sku Merchant'] = sehatq['Sku Merchant'].astype(str)

wms_sehatq = wms_sehatq.merge(sehatq[['Transaction id', 'Sku Merchant', 'Fee Rp', 'Transaction value']], how = 'left', left_on = ['Order #', 'Real SKU'], right_on = ['Transaction id', 'Sku Merchant'])
wms_sehatq = wms_sehatq.drop(['Transaction id', 'Sku Merchant'], axis = 1)

In [38]:
# wms_telkom_filt['Product Name']

In [39]:
## run 27
non_bundle = wms_sehatq[wms_sehatq['Bundle Name'].isnull()]
bundling =  wms_sehatq[wms_sehatq['Bundle Name'].notnull()]

non_bundle['Total Produk'] = non_bundle['Transaction value']
non_bundle['Total Produk Prop'] = non_bundle['Transaction value']
non_bundle['Comfee'] = non_bundle['Fee Rp']
non_bundle['Comfee Prop'] = non_bundle['Fee Rp']

bundling = bundling.drop(['Total Produk Prop'], axis = 1)
bundling = bundling.merge(non_bundle[['Order #', 'Product Name', 'Total Produk Prop','Comfee Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])

bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling['Comfee Prop'] = bundling['Comfee Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling = bundling.drop('Product Name_y', axis = 1).rename(columns = {'Product Name_x' : 'Product Name'})

# wms_sehatq = non_bundle.append(bundling, ignore_index = True, sort = False)

In [40]:
# run 28
wms_sehatq = non_bundle.append(bundling, ignore_index = True, sort = False)

In [41]:
## run 29
wms_sehatq = wms_sehatq.drop(['Fee Rp', 'Transaction value'], axis = 1)

In [42]:
## run 30
wms_sehatq['Comfee Prop'] = wms_sehatq['Comfee Prop'] * (-1)

In [43]:
## run 31
wms_telkom_filt = wms_telkom_filt[wms_telkom_filt['Store'] != 'Sehatq']
wms_telkom_filt = wms_telkom_filt.append(wms_sehatq, ignore_index = True, sort = False)

# Nutrimart WA

In [44]:
## run 32
wa22 = pd.read_excel(r"Settlement Project\Manual\Settlement YM - Xendit YM -ok.xlsx")
wa23 = pd.read_excel(r"Settlement Project\Manual\11. Settlement Xendit_YM 2023.xlsx")
# wa23.columns= wa23.columns.str.lower()

wa=pd.concat([wa22,wa23],ignore_index=True)

wa = wa.rename(columns = {
    'Transaction Type' : 'type',
})

wa.columns = [x.lower() for x in wa.columns]

In [45]:
## run 33
ref = pd.read_excel(r"Settlement Project\Manual\Wa Set Reference.xlsx")
ref=ref.drop_duplicates()

In [46]:
## run 34
for i in wa.columns:
    if 'Unnamed' in i:
        wa = wa.drop(i, axis = 1)
wa

,no,product id,transaction id,type,line type,payment channel,reference,currency,amount,balance,debit or credit,created date iso,time zone,created date,payment date,settlement date,completed date,bank code,account number,channel reference,name,description,invoice id,unnamed: 23,tanggal transaksi,keterangan,cabang,jumlah,cek,final cek,ledger sequence,unnamed: 24
0,30,61c877ee02b6d17f93374137,c83700d1-1e61-49ec-8859-5d2a1a20eb64,INVOICE,TRANSACTION,MANDIRI,WA-1640527247,IDR,1700,1700,CREDIT,2021-12-26T14:10:55.903Z,+0700 UTC,"26 Dec 2021, 21:10:55","26 Dec 2021, 21:10:54","26 Dec 2021, 21:10:57","26 Dec 2021, 21:10:57",NaN,NaN,NaN,NaN,NaN,61c87591a5cc58f39de4fe4c,NaN,2022-01-07 00:00:00,SWITCHING CR TRANSFER DR 523 SNRNutrimart J...,998.0,"573,100.00 CR",WA-1640527247,NaN,NaN,NaN
1,29,61c877ee02b6d17f93374137,c83700d1-1e61-49ec-8859-5d2a1a20eb64,INVOICE Fee,FEE,NaN,WA-1640527247,IDR,3850,-2150,DEBIT,2021-12-26T14:10:55.903Z,+0700 UTC,"26 Dec 2021, 21:10:55",NaN,NaN,"26 Dec 2021, 21:10:58",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-07 00:00:00,SWITCHING CR TRANSFER DR 523 SNRNutrimart J...,998.0,"573,100.00 CR",WA-1640527247,NaN,NaN,NaN
2,28,61cc6c46cb2ae3ac863b84df,aa0d2179-712d-4336-9c98-c70cd3db25fd,INVOICE,TRANSACTION,MANDIRI,WA-1640751182,IDR,209000,206850,CREDIT,2021-12-29T14:10:16.200Z,+0700 UTC,"29 Dec 2021, 21:10:16","29 Dec 2021, 21:10:14","29 Dec 2021, 21:10:17","29 Dec 2021, 21:10:17",NaN,NaN,NaN,NaN,NaN,61cbe050f68103763c51563a,NaN,2022-01-07 00:00:00,SWITCHING CR TRANSFER DR 523 SNRNutrimart J...,998.0,"573,100.00 CR",NaN,NaN,NaN,NaN
3,27,61cc6c46cb2ae3ac863b84df,aa0d2179-712d-4336-9c98-c70cd3db25fd,INVOICE Fee,FEE,NaN,WA-1640751182,IDR,3850,203000,DEBIT,2021-12-29T14:10:16.200Z,+0700 UTC,"29 Dec 2021, 21:10:16",NaN,NaN,"29 Dec 2021, 21:10:19",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-07 00:00:00,SWITCHING CR TRANSFER DR 523 SNRNutrimart J...,998.0,"573,100.00 CR",NaN,NaN,NaN,NaN
4,26,61d0040ff9d6746fc1be2a38,f363bfd6-9732-44da-9678-d371c28fa0e6,INVOICE,TRANSACTION,MANDIRI,WA-1641022252,IDR,209400,412400,CREDIT,2022-01-01T07:34:40.746Z,+0700 UTC,"01 Jan 2022, 14:34:40","01 Jan 2022, 14:34:39","01 Jan 2022, 14:34:42","01 Jan 2022, 14:34:42",NaN,NaN,NaN,NaN,NaN,61d0032e6acdd9669d028887,NaN,2022-01-07 00:00:00,SWITCHING CR TRANSFER DR 523 SNRNutrimart J...,998.0,"573,100.00 CR",WA-1641022252,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1,disb-34713337-9a43-4e08-9414-eb799d98e99a,d0d6cf03-ad0c-4d46-9311-77aed4ede174,WITHDRAWAL,TRANSACTION,NaN,wid-5c618ab1-b19d-4c25-ba94-e651929dce4a,IDR,1663115,0,DEBIT,2022-09-22T06:14:17.467Z,'+0700 UTC,"22 Sep 2022, 13:14:17","22 Sep 2022, 13:14:17",NaN,"22 Sep 2022, 13:14:22",ID_BCA,943026006,NaN,NaN,NaN,NaN,-1663115.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,4,63f18a5a5d1775115503e901,33bd06be-d4ba-491b-8e39-2945c18a6560,VA_PAYMENT,TRANSACTION,BRI,WA-1676773837,IDR,97000,97000,CREDIT,2023-02-19T02:32:58.476Z,'+0700 UTC,"19 Feb 2023, 09:32:58","19 Feb 2023, 09:32:56","19 Feb 2023, 09:32:58","19 Feb 2023, 09:32:58",BRI,9.20017e+14,NaN,Nutrimart,Checkout Nutrimart WA-1676773837,63f189d03800e8bb2fcb884b,NaN,PEND,TRSF E-BANKING CR 2502/FTSCY/WS95051 93115.00 ...,0.0,"93,115.00 CR",NaN,NaN,2.468053e+09,NaN
102,3,63f18a5a5d1775115503e901,33bd06be-d4ba-491b-8e39-2945c18a6560,VA_PAYMENT Fee,FEE,NaN,WA-1676773837,IDR,3500,93500,DEBIT,2023-02-19T02:32:58.476Z,'+0700 UTC,"19 Feb 2023, 09:32:58",NaN,NaN,"19 Feb 2023, 09:32:58",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PEND,TRSF E-BANKING CR 2502/FTSCY/WS95051 93115.00 ...,0.0,"93,115.00 CR",NaN,NaN,2.468053e+09,NaN
103,2,63f18a5a5d1775115503e901,33bd06be-d4ba-491b-8e39-2945c18a6560,VA_PAYMENT VAT,VAT,NaN,WA-1676773837,IDR,385,93115,DEBIT,2023-02-19T02:32:58.476Z,'+0700 UTC,"19 Feb 2023, 09:32:58",NaN,NaN,"19 Feb 2023, 09:32:58",NaN,NaN,NaN,NaN,NaN,NaN,NaN,PEND,TRSF E-BANKING CR 2502/FTSCY/WS95051 93115.00 ...,0.0,"93,115.00 CR",NaN,NaN,2.468053e+09,NaN


In [47]:
## run 35
wa_merge = wa.merge(ref, how = 'left', left_on = 'type', right_on = 'Nama')

In [48]:
## check must null
pd.DataFrame(wa_merge[wa_merge['Kategori'].isnull()]['type'].unique())
# wa_merge[wa_merge['Kategori'].isnull()]

,0


In [49]:
## run 36
index = wa_merge[wa_merge['Negatif'] == 'Negatif'].index.to_list()
wa_merge['amount'][index] = wa_merge['amount'][index] * -1 

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [50]:
## run 37
wa_pivot = pd.pivot_table(wa_merge, index = 'reference', columns = 'Kategori', values = 'amount', aggfunc=np.sum).reset_index()

In [51]:
## run 38
wa_pivot['Total Produk'] = wa_pivot['Total Transfer'] + wa_pivot['Payment Fee']

In [55]:
wa_pivot

Kategori,reference,Payment Fee,Total Transfer,Withdrawl,Total Produk
0,61d79a27109a1f001af95e91,NaN,NaN,573100.0,NaN
1,61e7b984be6a32001a23a1fe,NaN,NaN,204950.0,NaN
2,61fde58d57ecf7001b14340a,NaN,NaN,188550.0,NaN
3,62031874e70cce001cf48bce,NaN,NaN,639700.0,NaN
4,6209fbb8017f6b001c8dd572,NaN,NaN,529619.0,NaN
5,621722b3bc48de001b737cdf,NaN,NaN,197980.0,NaN
6,6231b53818eb0f001b5f2bcf,NaN,NaN,499873.0,NaN
7,624bf2bbc4147a542aeac252,NaN,NaN,384621.0,NaN
8,624fb7efea48240b332ebe8f,NaN,NaN,388055.0,NaN
9,625faf39c0156b56395159fb,NaN,NaN,1027545.0,NaN


In [52]:
## run 39
wms_wa = wms_telkom_filt[wms_telkom_filt['Store'] == 'Nutrimart-WA']

wms_wa = wms_wa.drop(['Product Name_x', 'Product Name_y'], axis = 1)

wms_wa['Order #'] = wms_wa['Order #'].astype(str)
wa_pivot['reference'] = wa_pivot['reference'].astype(str)

wms_wa = wms_wa.drop(['Total Produk'], axis = 1)
wms_wa = wms_wa.merge(wa_pivot[['reference', 'Payment Fee', 'Total Produk']], how = 'left', left_on = 'Order #', right_on = 'reference')

In [53]:
## run 40
transfer1 = pd.read_excel(r'Settlement Project\Manual/Transaksi YM Bank Transfer-ok.xlsx', sheet_name = '2021')
transfer2 = pd.read_excel(r'Settlement Project\Manual/Transaksi YM Bank Transfer-ok.xlsx', sheet_name = '2022')

## timo jan 23
if "IDOrder" in transfer2.columns:
    print('renama col idorder')
    transfer2=transfer2.rename(columns={'IDOrder':'ID Order'})
                                        
                                        
transfer2 = transfer2[~transfer2['ID Order'].isin(transfer1['ID Order'])]
#tambah aug 22
if 617 not in transfer2.columns:
    if 'Jumlah' in transfer2.columns:
        print('ya')
        transfer2=transfer2.rename(columns={'Jumlah':617})
#sampai sini tambah aug 22
transfer = transfer1.append(transfer2, ignore_index = True, sort = False)
transfer.loc[transfer['ID Order'].str.contains('\n',na=False),'ID Order']=transfer['ID Order'].str.replace('\n','') ### tambah aug 22

transfer = transfer[(transfer[617].notnull())&(transfer[617]!='Jumlah')]### ignore si jumlah
transfer['Transfer'] = pd.to_numeric(transfer[617].astype(str).str.replace('.00 CR','', regex = False).str.replace('.00','', regex = False).str.replace(',','', regex = False).str.replace(' ','', regex = False)).astype(int)
# cek=transfer.copy()
transfer = transfer.rename(columns = {'ID Order' : 'Order ID'})
transfer['Order ID'] = transfer['Order ID'].astype(str).str.replace('.0', '', regex = False)

wa_transfer = wms_wa[wms_wa['Order #'].isin(transfer['Order ID'])]
non_transfer = wms_wa[~wms_wa['Order #'].isin(transfer['Order ID'])]

wa_transfer['Payment Fee'] = np.nan

wa_real = wa_transfer[wa_transfer['Bundle Name'].isnull()]
wa_bundle = wa_transfer[wa_transfer['Bundle Name'].notnull()]

wa_real['Order #'] = wa_real['Order #'].astype(str)
transfer['Order ID'] = transfer['Order ID'].astype(str)
transfer = transfer.rename(columns =  {'Order ID' : 'Order #'})
transfer_short = transfer[['Order #', 'Transfer']].drop_duplicates('Order #')

wa_transfer = wa_real.merge(transfer_short, how = 'left', on = ['Order #'])
# wa_transfer['Transfer'] = wa_transfer['Transfer'].fillna(0) * wa_transfer['Total Net Before PPN']/wa_transfer.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
wa_transfer['Total Produk'] = wa_transfer['Transfer']

wa_aft_transfer = wa_transfer.append(non_transfer, ignore_index = True, sort = False)

renama col idorder
ya


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [54]:
## run 41
non_bundle = wa_aft_transfer[wa_aft_transfer['Bundle Name'].isnull()]
bundling = wa_aft_transfer[wa_aft_transfer['Bundle Name'].notnull()]

In [55]:
## run 42
non_bundle['Total Produk Prop'] = non_bundle['Total Produk'].fillna(0) * non_bundle['Total Net Before PPN']/non_bundle.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
non_bundle['Payment Fee Prop'] = non_bundle['Payment Fee'].fillna(0) * non_bundle['Total Net Before PPN']/non_bundle.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

In [56]:
## run 43
non_bundle = non_bundle.drop(['reference'], axis = 1)

In [57]:
## run 44
wa_jan = pd.read_excel("Settlement Project\Backup\Realisasi Jan 2022\Running Manual Jan 2022.xlsx")
wa_jan = wa_jan[(wa_jan['Channel'] == 'Nutrimart-WA') & (wa_jan['Paid'] == 'Paid')]
null_wa = non_bundle[(non_bundle['Total Produk'].isnull()) & (non_bundle['Order #'].isin(wa_jan['Order #']))]
null_wa = null_wa.drop(['Total Produk Prop', 'Payment Fee Prop', 'Total Produk'], axis = 1)

null_wa['Order #'] = null_wa['Order #'].astype(str)
null_wa['Real SKU'] = null_wa['Real SKU'].astype(str)

wa_jan['Order #'] = wa_jan['Order #'].astype(str)
wa_jan['Real SKU'] = wa_jan['Real SKU'].astype(str)

null_wa = null_wa.merge(wa_jan[['Order #', 'Real SKU', 'Bundle Name', 'Total Produk', 'Total Produk Prop', 'Payment Fee Prop']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', on = ['Order #', 'Real SKU', 'Bundle Name']).set_index(null_wa.index)

non_bundle['Total Produk'][null_wa.index] = null_wa['Total Produk']
non_bundle['Total Produk Prop'][null_wa.index] = null_wa['Total Produk Prop']
non_bundle['Payment Fee Prop'][null_wa.index] = null_wa['Payment Fee Prop']

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docu

In [58]:
## run 45
bundling = bundling.drop(['Total Produk Prop'], axis = 1)
bundling = bundling.merge(non_bundle[['Order #', 'Product Name', 'Total Produk Prop','Payment Fee Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])

bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling['Payment Fee Prop'] = bundling['Payment Fee Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling = bundling.drop('Product Name_y', axis = 1).rename(columns = {'Product Name_x' : 'Product Name'})


In [59]:
## run 46
wa_aft_transfer = non_bundle.append(bundling, ignore_index = True, sort = False)

In [60]:
## run 47
wms_telkom_filt = wms_telkom_filt[wms_telkom_filt['Store'] != 'Nutrimart-WA']
wms_telkom_filt = wms_telkom_filt.append(wa_aft_transfer, ignore_index = True, sort = False)

# Grab Mart

In [61]:
## run 48
grab22 = pd.read_excel(r"Settlement Project\Manual\Settlement GrabMart 2022 - ok.xlsx")
grab23= pd.read_excel("Settlement Project\Manual\Settlement GrabMart.xlsx")

grab=grab22[grab23.columns].append(grab23,ignore_index=True)

for i in grab.columns:
    if 'Unnamed' in i:
        grab = grab.drop(i, axis = 1)
grab['No Pesanan'] = grab['No Pesanan'].astype(str).str.replace('-', ' ')
grab['No Pesanan'] = grab['No Pesanan'].astype(str).str.split(' ', expand = True)[0] + ' ' + grab['No Pesanan'].astype(str).str.split(' ', expand = True)[1]
# # grab

In [62]:
## run 48 tambahan
grab.loc[(grab['No Pesanan']=='GM 057')&
         (grab['Month']=='Oktober')&
        (grab['Date']==21),'No Pesanan']=grab['No Pesanan']+"a"
grab.loc[(grab['No Pesanan']=='GM 362')&
         (grab['Month']=='Juni')&
        (grab['Date']==23),'No Pesanan']=grab['No Pesanan']+"a"
grab.loc[(grab['No Pesanan']=='GM 265')&
         (grab['Month']=='Maret')&
        (grab['Date']==28),'No Pesanan']=grab['No Pesanan']+"a"

In [63]:
## run 49 harus kosong
print('duplicate di settlement')
cek=grab.groupby(['No Pesanan'])[['Nilai Pesanan']].nunique().sort_values('Nilai Pesanan',ascending=False)
display(cek[cek['Nilai Pesanan']>1].reset_index())

duplicate di settlement


,No Pesanan,Nilai Pesanan


In [64]:
## run 50
wms_grab = wms_telkom_filt[wms_telkom_filt['Store'] == 'Grab Mart']
wms_grab = wms_grab[wms_grab['Order Status'] != 'cancelled'] # tambahan timo jan 23

wms_grab = wms_grab.drop(['Product Name_x', 'Product Name_y'], axis = 1)

indeks = wms_grab[~wms_grab['Order #'].astype(str).str.contains('^GM')].index.to_list()
wms_grab['Order #'][indeks] = wms_grab['Order #'][indeks].astype(str).str.split(n = 1).str[1].str.replace('-', ' ')

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [65]:
# run 50 tambahan , harus allign sama run 48 tambahan
wms_grab.loc[(wms_grab['Order #']=='GM 057')&
         (wms_grab['Month']=='October')&
        (wms_grab['Date']==20),'Order #']=wms_grab['Order #']+"a"
wms_grab.loc[(wms_grab['Order #']=='GM 265')&
         (wms_grab['Month']=='March')&
        (wms_grab['Date']==28),'Order #']=wms_grab['Order #']+"a"

In [66]:
### run 51
print('duplicate di wms')
cek=wms_grab.groupby(['Order #'])[['True datetime']].nunique()#.sort_values('True datetime',ascending=False)
display(cek[cek['True datetime']>1])
# cek

duplicate di wms


,True datetime
Order #,


In [67]:
### run 52
min1 = wms_grab[wms_grab['Order #'].astype(str).str.contains('_1')]
min1['Order #'] = min1['Order #'].astype(str).str.replace('_1', '', regex = False)

temp_telkom = wms_grab.merge(min1, how = 'left', on = ['Order #', 'Real SKU']).set_index(wms_grab.index)
indeks = temp_telkom[temp_telkom['Order Status_y'].notnull()].index.to_list()
wms_grab = wms_grab.drop(indeks, axis = 0)

wms_grab['Order #'] = wms_grab['Order #'].astype(str).str.replace('_1', '', regex = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
## run 53
wms_grab = wms_grab.merge(grab[['No Pesanan', 'Total Komisi & Pajak', 'Total Pendapatan']], how = 'left', left_on = 'Order #', right_on = 'No Pesanan')

In [69]:
## run 54
wms_grab['Total Produk'] = wms_grab['Total Pendapatan']
wms_grab['Comfee'] = wms_grab['Total Komisi & Pajak']


In [70]:
## run 55
non_bundle = wms_grab[wms_grab['Bundle Name'].isnull()]
bundling = wms_grab[wms_grab['Bundle Name'].notnull()]

In [71]:
## run 56
non_bundle.loc[non_bundle['Comfee']=='10293,70','Comfee']=10293.7

In [72]:
## run 57
non_bundle['Total Produk Prop'] = non_bundle['Total Produk'].fillna(0) * non_bundle['Total Net Before PPN']/non_bundle.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
non_bundle['Comfee Prop'] = non_bundle['Comfee'].fillna(0) * non_bundle['Total Net Before PPN']/non_bundle.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

In [73]:
## run 58
bundling = bundling.drop(['Total Produk Prop', 'Comfee Prop'], axis = 1)
bundling = bundling.merge(non_bundle[['Order #', 'Product Name', 'Total Produk Prop','Comfee Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])

bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling['Payment Fee Prop'] = bundling['Comfee Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling = bundling.drop('Product Name_y', axis = 1).rename(columns = {'Product Name_x' : 'Product Name'})


In [74]:
## ru 59
wms_grab = non_bundle.append(bundling, ignore_index = True, sort = False)

In [75]:
## run 60
wms_grab = wms_grab.drop(['No Pesanan', 'Total Komisi & Pajak', 'Total Pendapatan'], axis = 1)

In [76]:
## run 61
wms_grab['Payment Fee Prop'] = wms_grab['Payment Fee Prop'] * (-1)

In [77]:
## run 62
wms_telkom_filt['Channel'].unique()

array(['Grab Mart', 'Orami', 'IndoHp', 'Evermos', 'Sehatq',
       'Nutrimart-WA'], dtype=object)

In [78]:
## run 63
wms_telkom_filt = wms_telkom_filt[wms_telkom_filt['Store'] != 'Grab Mart']
wms_telkom_filt = wms_telkom_filt.append(wms_grab, ignore_index = True, sort = False)

# Orami

In [79]:
## run 64
orami22 = pd.read_excel(r'Settlement Project\Manual/Settlement Orami 2022 - ok.xlsx')

orami23=pd.read_excel(r"Settlement Project\Manual\05. Settlement Orami 2023.xlsx")
if 'Amount' in orami23.columns:
    orami23=orami23.rename(columns={'Amount':'Total Transfer'})
orami23[['Tahun','Bulan','Tanggal','Type Transaksi','Keterangan','Total Transfer']]

orami=orami22.append(orami23,ignore_index=True)

orami = orami[orami['Type Transaksi'] == 'Penghasilan']
orami['Order ID'] = orami['Keterangan'].astype(str).str.split('#', expand = True)[1].str.replace('.0', '', regex = False)

In [80]:

# orami

In [81]:
## run 65
# wms_orami = wms_telkom_filt[wms_telkom_filt['Store'] == 'Orami']
wms_orami = wms_telkom_filt[(wms_telkom_filt['Store'] == 'Orami')&(wms_telkom_filt['Order Status']!='cancelled')] ## tambahan timo nov 2022
wms_orami = wms_orami.drop(['Product Name_x', 'Product Name_y'], axis = 1)

In [82]:
## run 66
min1 = wms_orami[wms_orami['Order #'].astype(str).str.contains('_1')]
min1['Order #'] = min1['Order #'].astype(str).str.replace('_1', '', regex = False)

temp_telkom = wms_orami.merge(min1, how = 'left', on = ['Order #', 'Real SKU']).set_index(wms_orami.index)
indeks = temp_telkom[temp_telkom['Order Status_y'].notnull()].index.to_list()
wms_orami = wms_orami.drop(indeks, axis = 0)

wms_orami['Order #'] = wms_orami['Order #'].astype(str).str.replace('_1', '', regex = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [83]:
## run 67
min1 = wms_orami[wms_orami['Order #'].astype(str).str.contains('-1')]
min1['Order #'] = min1['Order #'].astype(str).str.replace('-1', '', regex = False)

temp_telkom = wms_orami.merge(min1, how = 'left', on = ['Order #', 'Real SKU']).set_index(wms_orami.index)
indeks = temp_telkom[temp_telkom['Order Status_y'].notnull()].index.to_list()
wms_orami = wms_orami.drop(indeks, axis = 0)

wms_orami['Order #'] = wms_orami['Order #'].astype(str).str.replace('-1', '', regex = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
## run 68
wms_orami = wms_orami.merge(orami[['Keterangan', 'Order ID', 'Total Transfer']], how = 'left', left_on = 'Order #', right_on = 'Order ID')
wms_orami['Total Produk'] = wms_orami['Total Transfer'] ###knp error ya


In [85]:
## run 69
non_bundle = wms_orami[wms_orami['Bundle Name'].isnull()]
bundling = wms_orami[wms_orami['Bundle Name'].notnull()]

In [86]:
## run 70
non_bundle['Total Produk Prop'] = non_bundle['Total Produk'].fillna(0) * non_bundle['Total Net Before PPN']/non_bundle.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

non_bundle = non_bundle.drop(['Keterangan', 'Order ID', 'Total Transfer'], axis = 1)

bundling = bundling.drop(['Total Produk Prop'], axis = 1)
bundling = bundling.merge(non_bundle[['Order #', 'Product Name', 'Total Produk Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])

bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling = bundling.drop('Product Name_y', axis = 1).rename(columns = {'Product Name_x' : 'Product Name'})

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
## run 71
wms_orami = non_bundle.append(bundling, ignore_index = True, sort = False)

In [88]:
## run 72
wms_telkom_filt = wms_telkom_filt[wms_telkom_filt['Store'] != 'Orami']
wms_telkom_filt = wms_telkom_filt.append(wms_orami, ignore_index = True, sort = False)

# Finalize

In [89]:
### run 73
wms_telkom_filt['Paid'] = np.nan

indeks = wms_telkom_filt[
    (wms_telkom_filt['Order #'].isin(indohp['No Order'])) |
    (wms_telkom_filt['Order #'].isin(sehatq['Transaction id'])) |
    (wms_telkom_filt['Order #'].isin(wa_pivot['reference'])) |
    (wms_telkom_filt['Order #'].isin(transfer['Order #'])) |
    (wms_telkom_filt['Order #'].isin(grab['No Pesanan'])) |
    (wms_telkom_filt['Order #'].isin(orami['Order ID'])) |
    (wms_telkom_filt['Order #'].isin(wa_jan['Order #']))|
    (wms_telkom_filt['Order #'].isin(evermos['ID Order']))
].index.to_list()

wms_telkom_filt['Paid'][indeks] = 'Paid'

tanpa_bayar = pd.read_excel(r'Settlement Project\Manual/Transaksi YM Bank Transfer-ok.xlsx', sheet_name = 'Rekap Tanpa Bayar')

print(len(tanpa_bayar))

indeks = wms_telkom_filt[wms_telkom_filt['Order #'].isin(tanpa_bayar['ID Order'])].index.to_list()
wms_telkom_filt['Paid'][indeks] = 'Paid'

print(len(indeks))

cancel = pd.read_excel(r'Settlement Project\Manual/Transaksi YM Bank Transfer-ok.xlsx', sheet_name = 'Transaksi Cancel')

print(len(cancel))
indeks = wms_telkom_filt[wms_telkom_filt['Order #'].isin(cancel['Order #'])].index.to_list()
wms_telkom_filt['Paid'][indeks] = 'Cancel'

print(len(indeks))

wms_telkom_filt['Paid'] = wms_telkom_filt['Paid'].fillna('Unpaid')

wms_telkom_filt['Total Produk Prop Adj'] = wms_telkom_filt['Total Produk Prop']/1.11
wms_telkom_filt['BTL Cost'] = (wms_telkom_filt['Total Produk Prop Adj']) - wms_telkom_filt['Total Net Before PPN']

1
2
2
0


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docu

In [90]:
## run 74
wms_telkom_filt['Order #'] = wms_telkom_filt['Order #'].astype(str)

In [112]:
# wms_telkom_filt

In [91]:
## run 75
import pandas as pd

prev = pd.read_csv(r"D:\6. settlement\Code\Settlement Project\Backup\Realisasi Apr 2023\Final Result\Raw Unpivoted\CSV\Running Manual Apr 23.csv", sep = ';')
prev_month = 'April'
prev = prev.rename(columns = {'Paid' : 'Paid ' + prev_month})

#tambahan timo nov 2022
# prev.loc[prev['Order #'].str.contains('\n'),'Order #']=prev['Order #'].str.replace('\n','')

# prev['Qty. Invoiced'] = prev['Qty. Invoiced'].astype(float)
current = wms_telkom_filt.copy()
# current = pd.read_csv("D:\Backup\Backup 26 Mei\Settlement Project Backup\Realisasi Januari 2022\Final Result\Running Nutrimart Jan Jan 2022.csv", sep = ';')

prev['Order #'] = prev['Order #'].astype(str)
prev['Real SKU'] = prev['Real SKU'].astype(str)
# display(prev[prev['Order #']=='13769264'])
current['Order #'] = current['Order #'].astype(str)
current['Real SKU'] = current['Real SKU'].astype(str)

prev['Bundle Name'] = prev['Bundle Name'].astype(str)
indeks = prev[prev['Bundle Name'] == 'nan'].index.to_list()
prev['Bundle Name'][indeks] = np.nan

merged = current.merge(prev[['Order #','Real SKU', 'Paid ' + prev_month, 'Bundle Name']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', on = ['Order #', 'Real SKU', 'Bundle Name'])
import numpy as np

merged['Paid Beda'] = np.nan

indeks = merged[merged['Paid'] != merged['Paid ' + prev_month]].index.to_list()
merged['Paid Beda'][indeks] = 'Beda'

merged['Paid Beda'] = merged['Paid Beda'].fillna('Sama')
# merged[merged['Paid Beda'] == 'Beda'][['Order #','Brand','Real SKU','Month', 'Paid_x', 'Paid_y', 'Paid Beda']]
merged[merged['Paid Beda'] == 'Beda'].groupby(['Year', 'Month','Store', 'Paid', 'Paid ' + prev_month], dropna = False)['Order #'].nunique()

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Year  Month  Store         Paid    Paid April
2023  April  Orami         Paid    Unpaid        1
                                   NaN           3
      March  Grab Mart     Unpaid  NaN           2
      May    Grab Mart     Unpaid  NaN           1
             Nutrimart-WA  Unpaid  NaN           1
             Orami         Paid    NaN           4
                           Unpaid  NaN           2
Name: Order #, dtype: int64

In [92]:

temp = prev[['Order #','Real SKU', 'Paid ' + prev_month, 'Bundle Name']].applymap(type)

for i in temp.columns :
    print(i)
    print(temp[i].unique())

Order #
[<class 'str'>]
Real SKU
[<class 'str'>]
Paid April
[<class 'str'>]
Bundle Name
[<class 'float'> <class 'str'>]


In [93]:
## run 76
merged = merged.rename(columns = {'Paid' : 'Paid May'})
# merged.to_csv('D:\settlement\Code\Settlement Project\Backup\Realisasi Mei 2022\Final Result\Raw Unpivoted\CSV\Running Manual Mei 22.csv', sep = ';', index = False)

In [95]:
## run 77
finres=pd.read_excel("Settlement Project\Backup\Realisasi Apr 2023\Final Result\Raw Unpivoted\Excel\Running Manual Apr 23.xlsx")

In [96]:
## run 78
finres=finres.rename(columns={'Paid April':'Paid May'})
finres=finres.rename(columns={'Paid March':'Paid April'})




In [97]:
## run 79
merged[finres.columns].to_excel('Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\Excel\Running Manual May 23.xlsx', index = False)

In [98]:
## run 80
merged[finres.columns].to_csv('Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\CSV\Running Manual May 23.csv',sep=';', index = False)

In [ ]:
#### selesai ####

In [213]:
transfer2 = pd.read_excel(r'Settlement Project\Manual/Transaksi YM Bank Transfer.xlsx', sheet_name = '2022', keep_default_na=False)
transfer2

,Tgl,Mutasi,Bank,617,Nama Konsumen,ID Order,ID order Lama,Note,Tanggal Proses,Area,Code Area
0,2022-03-01 00:00:00,NUTRIMART,0,765400,Rifin Hartono,WA-1641201239,,,2021-04-01 00:00:00,ECOM NFI- YELLOW MESSENGER,A50061
1,17/01/2022,NUTRIMART IDR,0,486000,Sri Rangan,WA-1642378145,,,17/01/2022,ECOM NFI- YELLOW MESSENGER,A50061
2,21/01/2022,NUTRIMART,0,371700,Syane,WA-1642746820,,,21/01/2022,ECOM NFI- YELLOW MESSENGER,A50061
3,24/01/2022,NUTRIMART,0,120000,Lie Ratnawati,WA-1643003375,,,24/01/2022,ECOM NFI- YELLOW MESSENGER,A50061
4,26/01/2022,NUTRIMART IDR,0,258200,Natalya Kaseger,WA-1643189695,,,27/01/2021,ECOM NFI- YELLOW MESSENGER,A50061
5,27/01/2022,NUTRIMART IDR,0,148400,Audreyana Sobana,WA-1643243780,,,27/01/2021,ECOM NFI- YELLOW MESSENGER,A50061
6,2022-01-02 00:00:00,NUTRIMART IDR,0,286000,Lita Herlina,WA-1643765690,,,2021-02-02 00:00:00,ECOM NFI- YELLOW MESSENGER,A50061
7,2022-04-02 00:00:00,NUTRIMART,0,172530,Natalya Kaseger,WA-1643955304,,,2022-05-02 00:00:00,ECOM NFI- YELLOW MESSENGER,A50061
8,2022-07-02 00:00:00,NUTRIMART IDR,,223500,Adissa Adissa,1000098867,,Cancel berubah ID ke Web NM,2022-07-02 00:00:00,ECOM NFI- YELLOW MESSENGER,A50061
9,2022-07-02 00:00:00,NUTRIMART IDR,,396780,Tina Adelina,WA-1644215159,,,2022-08-02 00:00:00,ECOM NFI- YELLOW MESSENGER,A50061


# Tiktok

In [1]:
## run -1

import pandas as pd
import numpy as np
import os

tiktok = pd.DataFrame()
for i in os.listdir(r'Settlement Project/Manual/Tiktok/Seller Centre/'):
    temp = pd.read_excel(r'Settlement Project/Manual/Tiktok/Seller Centre/' + str(i), sheet_name = 'OrderSKUList')
    temp['Source'] = i
    print(i)
    true_col = ['Order ID', 'Order Status', 'Order Substatus',
       'Cancelation/Return Type', 'Normal or Pre-order', 'SKU ID',
       'Seller SKU', 'Product Name', 'Variation', 'Quantity',
       'Sku Quantity of return', 'SKU Unit Original Price',
       'SKU Subtotal Before Discount', 'SKU Platform Discount',
       'SKU Seller Discount', 'SKU Subtotal After Discount',
       'Shipping Fee After Discount', 'Original Shipping Fee',
       'Shipping Fee Seller Discount', 'Shipping Fee Platform Discount',
       'Taxes', 'Order Amount', 'Order Refund Amount', 'Created Time',
       'Paid Time', 'RTS Time', 'Shipped Time', 'Delivered Time',
       'Cancelled Time', 'Cancel By', 'Cancel Reason', 'Fulfillment Type',
       'Warehouse Name', 'Tracking ID', 'Delivery Option',
       'Shipping Provider Name', 'Buyer Message', 'Buyer Username',
       'Recipient', 'Phone #', 'Zipcode', 'Country', 'Province',
       'Regency and City', 'Districts', 'Villages', 'Detail Address',
       'Payment Method', 'Weight(kg)', 'Product Category',
       'Shipping Label Print Status', 'Picking List Print Status', 'Source']
    temp = temp[temp['Order ID'] != 'Platform unique order ID.']
    if len(tiktok) != 0:
        temp = temp[~temp['Order ID'].isin(tiktok['Order ID'])]
    tiktok = tiktok.append(temp, ignore_index = True, sort = False)
    if temp.columns.to_list() == true_col:
        print('Column Sama')
    else :
        print('Column Beda')
        print([x for x in true_col if x not in temp.columns.to_list()])
        print([x for x in temp.columns.to_list() if x not in true_col])

Orders-2021-09-01_2021-11-30.xlsx
Column Sama
Orders-2021-12-01_2021-12-31.xlsx
Column Sama
Orders-2022-01-01_2022-01-31.xlsx
Column Beda
[]
['Additional address information']
Orders-2022-02-01_2022-02-28.xlsx
Column Beda
['Shipping Label Print Status', 'Picking List Print Status']
['Additional address information', 'Package ID']
Orders-2022-03-01_2022-03-31.xlsx
Column Beda
['Shipping Label Print Status', 'Picking List Print Status']
['Additional address information', 'Package ID', 'Seller Note']
Orders-2022-04-01_2022-04-25.xlsx
Column Beda
['Shipping Label Print Status', 'Picking List Print Status']
['Additional address information', 'Package ID', 'Seller Note']
Orders-2022-04-26_2022-04-30.xlsx
Column Beda
['Shipping Label Print Status', 'Picking List Print Status']
['Additional address information', 'Package ID', 'Seller Note']
Orders-2022-05-01_2022-05-23.xlsx
Column Beda
['Shipping Label Print Status', 'Picking List Print Status']
['Additional address information', 'Package ID',

In [2]:
## for backup
tiktok_ori=tiktok.copy()

In [17]:
## for backup
tiktok=tiktok_ori.copy()

In [3]:
### run - 2
### komen 15 aug
tiktok = tiktok.rename(columns = {
    'Order ID' : 'Order #',
    'Seller SKU' : 'SKU',
    'Quantity' : 'Qty. Invoiced',
    'Created Time' : 'Order date',
    'Paid Time' : 'Paid Date',
    'Tracking ID' : 'AWB',
    'Shipping Provider Name' : 'Shipping Courier',
    'Province' : 'Region Group'
})


### new 15 aug
# tiktok = tiktok.rename(columns = {
#     'Order ID' : 'Order #',
#     'Seller SKU' : 'SKU',
#     'Quantity' : 'Qty. Invoiced',
#     'Created Time' : 'Order date',
#     'Paid Time' : 'Paid Date',
#     'Tracking ID' : 'AWB',
#     'Shipping Provider Name' : 'Shipping Courier',
#     'Province' : 'Region Group',
#     'Package ID' : 'Order # WMS',
# })

tiktok['Sales Order ID'] = tiktok['Order #']
tiktok['SKU'] = tiktok['SKU'].astype(str).str.replace('\t', '', regex = False)

In [4]:
### run -3
tiktok = tiktok[[x for x in tiktok.columns if x in ['Order #',
 'Sales Order ID',
 'AWB',
 'Paid Date',
 'Order date',
 'Week',
 'Date',
 'Month',
 'Quarter',
 'Year',
 'Channel',
 'SKU',
 'Brand',
 'Product Name',
 'Bundle Name',
 'Price List NFI',
 'Qty. Invoiced',
 'Total Net',
 'Sub Brand',
 'Real SKU',
 'Real Nama Produk',
 'Parent Item',
 'Parent SKU',
 'Bundle Flag',
 'Shipping Courier',
 'Store',
 'True datetime',
 'Store Type',
 'Region Group',
 'PL Before PPN',
 'Total Net Before PPN',
 'Category Baru',
 'Exported Parent Item',
 'Order Date Seller Centre',
 'Product ID',
 'Total Selcen',
 'Order Status',
 'Warehouse Origin',
 'Paid',
 'SO STATUS',
 'WH',
 'Warehouse Quantity',
 'Total Produk Prop',
 'Fullfee Prop',
 'Comfee Prop',
 'BTL Cost',
 'Total Produk Prop Adj','Package ID'     
 ]]]

In [5]:
### run 3.1
tiktok['true datetime']=pd.to_datetime(pd.to_datetime(tiktok['Order date']).dt.strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
### run -4
temp2 = tiktok.copy()

import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os
    
data_SKU = pd.read_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx")

s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

to_excel = data_SKU.to_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx", index = False)

data_SKU.loc[data_SKU['Price List NFI']=='-','Price List NFI']=0
data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')

data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

In [7]:
### run 4.1
dfalias=pd.DataFrame()
for i in [x for x in data_SKU if 'Alias SKU' in x]:
    dfaliastemp=data_SKU[data_SKU[i].notnull()][['Real SKU',i]].rename(columns={i:'SKU Alias'})
    # dfaliastemp['SKU Alias']=dfaliastemp['SKU Alias'].astype(str)
    dfalias=dfalias.append(dfaliastemp)
cek=temp2[~temp2['SKU'].isin(data_SKU['Real SKU'].unique())]['SKU'].unique()
# dfalias[dfalias['SKU Alias'].isin(cek)]
for i in cek:
    skualias=dfalias[dfalias['SKU Alias']==i]['Real SKU'].unique()[0]
    temp2.loc[temp2['SKU']==i,'SKU']=skualias
    # print(skualias)

In [8]:
### run 4.2

from datetime import datetime,timedelta
listjanjian=requests.get('http://event.l-men.com/apiv1?type=janjian')
for i in listjanjian.json():
    sku=i['realsku']
    print(sku)
    start=datetime.fromtimestamp(int(i['startdate']/1000)).replace(hour=0)
    end=datetime.fromtimestamp(int(i['enddate']/1000)).replace(hour=23)
    print(i['notes'])
    temp2.loc[(temp2['SKU']==sku)&(temp2['true datetime']>=start)&(temp2['true datetime']<end),'Pemilik Budget']=i['notes']
    print('=====')


2T00803164
Launching tahap 2
=====
2T00803164P3
Launching tahap 2
=====
2T02703229P2
Launching
=====
PT25T74
Launching
=====
2T02703229
Launching tahap 2
=====
2104115110
Launching tahap 2
=====
2104115110P2
Launching tahap 2
=====
2104148024
Launching tahap 2
=====
2104148024P2
Launching tahap 2
=====
1N01402250P24
Marketing
=====
1N01402250P4
Marketing
=====
2104010270
Marketing
=====
1101909331
Marketing
=====
2T00803164P2
Marketing
=====
2T00803164P3
Marketing
=====
2HF0114021
Launching tahap 2
=====
2101751112
Marketing
=====
2101751112P2
Marketing
=====
2101700166P2
Marketing
=====
2101700166
Marketing
=====
2101384106
Internal
=====
2101384106P3
Internal
=====
2105055180P2
Internal
=====
2104523105P3
Internal
=====
2104508105P3
Internal
=====
2104543105P3
Internal
=====
2104589105P3
Internal
=====
2104241210P2
Internal
=====
2106386249P24
Internal
=====
2101000195
Internal
=====
2101151180
Internal
=====
2101150180
Internal
=====
2101200180
Internal
=====
2102500125
Internal
===

In [9]:
## run 4.3
list_col = ['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand']

temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
for i in list_col:
    temp2[i] = temp_merge[str(i)]

list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(temp2.index)
list_pcs = [x for x in temp_merge.columns if 'PCS' in x]
for i in list_pcs:
    temp_merge[i] = temp_merge[i].fillna(0).astype(int) * temp_merge['Qty. Invoiced'].astype(int)

list_col = data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()    
for i in list_col:
    temp2[i] = temp_merge[str(i)]
    
    
print("Unbundling")
data_bundle1 = temp2[~temp2['Produk 1'].isnull()]
data_bundle1['Bundle Name'] = data_bundle1['Product Name']
data_bundle1['Product Name'] = data_bundle1['Produk 1']
data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
data_bundle1['Bundle Flag'] = np.nan

data_bundle2 = temp2[~temp2['Produk 2'].isnull()]
data_bundle2['Bundle Name'] = data_bundle2['Product Name']
data_bundle2['Product Name'] = data_bundle2['Produk 2']
data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
data_bundle2['Bundle Flag'] = np.nan

data_bundle3 = temp2[~temp2['Produk 3'].isnull()]
data_bundle3['Bundle Name'] = data_bundle3['Product Name']
data_bundle3['Product Name'] = data_bundle3['Produk 3']
data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
data_bundle3['Bundle Flag'] = np.nan

data_bundle4 = temp2[~temp2['Produk 4'].isnull()]
data_bundle4['Bundle Name'] = data_bundle4['Product Name']
data_bundle4['Product Name'] = data_bundle4['Produk 4']
data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
data_bundle4['Bundle Flag'] = np.nan

data_bundle5 = temp2[~temp2['Produk 5'].isnull()]
data_bundle5['Bundle Name'] = data_bundle5['Product Name']
data_bundle5['Product Name'] = data_bundle5['Produk 5']
data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
data_bundle5['Bundle Flag'] = np.nan

data_bundle6 = temp2[~temp2['Produk 6'].isnull()]
data_bundle6['Bundle Name'] = data_bundle6['Product Name']
data_bundle6['Product Name'] = data_bundle6['Produk 6']
data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
data_bundle6['Bundle Flag'] = np.nan

data_bundle7 = temp2[~temp2['Produk 7'].isnull()]
data_bundle7['Bundle Name'] = data_bundle7['Product Name']
data_bundle7['Product Name'] = data_bundle7['Produk 7']
data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
data_bundle7['Bundle Flag'] = np.nan

data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
data_bundle['SKU'] = data_bundle['SKU'].astype(str)
data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item', 'Parent SKU']]

print("Pricing")
temp2 = temp2.append(data_bundle, ignore_index = True, sort = False)
list_col = ['Real SKU', 'Price List NFI', 'Harga Cost']

temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
for i in list_col:
    if i == 'Real SKU' or i == 'Price List NFI':
        temp2[i] = temp_merge[str(i) + '_y']
    else :
        temp2[i] = temp_merge[str(i)]
    
temp2['Price List NFI'] = pd.to_numeric(temp2['Price List NFI']).astype(int)
temp2['Harga Cost'] = pd.to_numeric(temp2['Harga Cost']).astype(int)
temp2['Qty. Invoiced'] = pd.to_numeric(temp2['Qty. Invoiced']).astype(int)

temp2['Total Net'] = temp2['Price List NFI'] * temp2['Qty. Invoiced']
temp2['Total Harga Cost'] = temp2['Harga Cost'] * temp2['Qty. Invoiced']

Unbundling


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

Pricing


In [10]:
## run -5
temp2[temp2['Price List NFI'].isnull()]['SKU']
## harus null

Series([], Name: SKU, dtype: object)

In [11]:
## run -6
temp2[temp2['Real SKU'].isnull()]
## harus null

,Order #,Order Status,SKU,Product Name,Qty. Invoiced,Order date,Paid Date,AWB,Shipping Courier,Region Group,...,Harga Cost 7,Harga Organik 7,Bundle Name,Price List NFI,Total Net,Bundle Flag,Parent Item,Parent SKU,Harga Cost,Total Harga Cost


In [12]:
## run 6.1 ### jan 23 timo
selcen_sku = temp2.copy()

In [13]:
## run 6.2 ### jan 23 timo
j_conv = pd.read_excel("Settlement Project\Rumus Realisasi.xlsx", sheet_name = 'Paket Brand')
j_conv = j_conv.iloc[:,:5]
j_conv = j_conv.rename(columns = {'SKU' : 'Real SKU', 'SKU.1' : 'SKU Produk'})
j_conv = j_conv[j_conv['Real SKU'].notnull()]
j_conv['SKU Produk'] = j_conv['SKU Produk'].astype(str).str.replace('.0', '',regex = False)

temp_all = selcen_sku.copy()

temp_all['Real SKU'] = temp_all['Real SKU'].astype(str)
j_conv['Real SKU'] = j_conv['Real SKU'].astype(str)

temp_all = temp_all.merge(j_conv, how = 'left', on = 'Real SKU')

paket_j = temp_all[temp_all['SKU Produk'].notnull()]

paket_j['Bundle Name'] = paket_j['Product Name']
paket_j['Product Name'] = paket_j['Name']
paket_j['SKU'] = paket_j['SKU Produk']
paket_j['Qty. Invoiced'] = paket_j['Qty. Invoiced'] * paket_j['Konversi ke UOM'].astype(float)
paket_j['Bundle Flag'] = np.nan

data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
if 'Real SKU_y' in paket_j.columns:
    paket_j = paket_j.drop('Real SKU_y', axis = 1)
paket_j['SKU'] = paket_j['SKU'].astype(str)
paket_j[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']] = paket_j.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(paket_j.index)[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y', 'Price List NFI_y']]

paket_j['Total Net'] = paket_j['Price List NFI'] * paket_j['Qty. Invoiced']

# for i in ['Biaya Pengiriman', 'Total Terbayar', 'Harga Produk']:
#     paket_j[i] = paket_j[i] * paket_j['Total Net']/paket_j.groupby(['ID Transaksi', 'Bundle Name'])['Total Net'].transform('sum')

paket_j = paket_j.drop(['Nama Paket', 'SKU Produk', 'Name', 'Konversi ke UOM'], axis = 1)
selcen_sku = selcen_sku.append(paket_j, ignore_index = True, sort = False)
indeks = selcen_sku[selcen_sku['Product Name'].isin(paket_j['Bundle Name'])].index.to_list()
selcen_sku['Brand'][indeks] = 'Bundle'

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [14]:
## run for check, jika ingin di unbundling isi rumus realisasi xlsx sheet paket brand
selcen_sku[(selcen_sku['Real SKU'].str.contains('J'))&~(selcen_sku['Real SKU'].isin(j_conv['Real SKU'].unique()))][['Real SKU','Real Nama Produk']].drop_duplicates()


,Real SKU,Real Nama Produk
86056,(J)71110138,Box Packing NutriMart


In [15]:
## run -7
# for_wms = temp2.copy() ## komen timo jan 23
for_wms = selcen_sku.copy()

In [16]:
## run -8
import pandas as pd
import numpy as np
import pypyodbc

conn = pypyodbc.connect('Driver={SQL Server};'
                        'Server=NFI-DB-01;'
                        'Database=gudang;'
                        # 'UID=andra.miftah;'
                        # 'PWD=24nutrifoodindonesia21;'
                        'Trusted_Connection=yes;'
                )

cursor = conn.cursor()
wms_telkom = pd.read_sql(r"""SELECT t1.id, t1.order_id,t1.status, t1.quantity, t1.product_id, t2.id,  t2.order_number, t2.invoice_number, t2.channel, t2.order_date, t3.id, t3.sku, t3.name
  FROM [gudang].[dbo].[outbound_items] as t1
  left join orders as t2
  on t1.order_id = t2.id
  left join products as t3
  on t1.product_id = t3.id
  where t2.channel = 'TikTok Commerce'""", conn)

print('pull from wms done')

wms_telkom_ori=wms_telkom.copy()


pull from wms done


In [10]:
### run -8 alternative
wms_telkom=pd.read_excel("D:\Downloads\Data Gudang 20221122 (1).xlsx")
wms_telkom=wms_telkom[wms_telkom['channel']=='TikTok Commerce']
wms_telkom_ori=wms_telkom.copy()

In [13]:
## for backup
wms_telkom=wms_telkom_ori.copy()

In [18]:
## run -9

# timo 19 aug
dfpackid=tiktok[tiktok['Package ID'].notnull()][['Order #','Package ID']].drop_duplicates().rename(columns={'Package ID':'invoice_number'})

for i in wms_telkom['invoice_number'].unique():
    # print(i)
    ordid=dfpackid[dfpackid['invoice_number']==i]['Order #'].unique()
    if len(ordid)!=0:
        wms_telkom.loc[wms_telkom['invoice_number']==i,'real order id']=ordid[0]
wms_telkom.loc[wms_telkom['real order id'].isnull(),'real order id']=wms_telkom['invoice_number']
print('done masking')

# wms_telkom = wms_telkom[wms_telkom['channel'] == 'TikTok Commerce']

# outbound_items = pd.read_csv(r'Backup Gudang 4 Sept 2021 Gudang Baru/outbound_items.csv', sep = ';')
# orders = pd.read_csv(r'Backup Gudang 4 Sept 2021 Gudang Baru/orders.csv', sep = ';')
# products = pd.read_csv(r'Backup Gudang 4 Sept 2021 Gudang Baru/products.csv', sep = ';')

# wms_telkom2 = outbound_items[['id', 'order_id','status', 'quantity', 'product_id']].merge(
#     orders[['id',  'order_number', 'invoice_number', 'channel', 'order_date']], how = 'left', left_on = 'order_id', right_on = 'id'
# ).merge(
#     products[['id', 'sku', 'name']], how = 'left', left_on = 'product_id', right_on = 'id'
# )
# wms_telkom2 = wms_telkom2[wms_telkom2['channel'].isin(['TikTok Commerce'])]


#timo nov 2022
if "STATUS" in wms_telkom.columns:
    wms_telkom=wms_telkom.rename(columns = {'STATUS' :'status'})
# wms_telkom = wms_telkom[['invoice_number', 'sku', 'status', 'quantity']] ### komen 19 aug
wms_telkom = wms_telkom[['real order id', 'sku', 'status', 'quantity']] ### timo 19 aug

# wms_telkom2 = wms_telkom2[['invoice_number', 'sku', 'status', 'quantity']].drop_duplicates()
# wms_telkom = wms_telkom.append(wms_telkom2, ignore_index = True, sort = False)

# wms_telkom = wms_telkom.groupby(['invoice_number', 'sku', 'status'])['quantity'].sum().reset_index()

wms_telkom['WH'] = 'Telkom'
# wms_telkom = wms_telkom.rename(columns = {'invoice_number' : 'Order #', 'status':'SO STATUS', 'sku' : 'SKU WH', 'quantity' : 'Warehouse Quantity'})  ### komen 19aug
wms_telkom = wms_telkom.rename(columns = {'real order id' : 'Order #', 'status':'SO STATUS', 'sku' : 'SKU WH', 'quantity' : 'Warehouse Quantity'})    ### new timo 19 aug

for_wms['Order #'] = for_wms['Order #'].astype(str)
wms_telkom['Order #'] = wms_telkom['Order #'].astype(str)

temp_telkom = for_wms[for_wms['Order #'].isin(wms_telkom['Order #'])]

# temp_telkom = temp_telkom.drop(['WH', 'Order #', 'SO STATUS', 'SKU WH', 'Warehouse Quantity'], axis = 1)
temp_telkom['Order #'] = temp_telkom['Order #'].astype(str)
wms_telkom['Order #'] = wms_telkom['Order #'].astype(str)

temp_telkom['Real SKU'] = temp_telkom['Real SKU'].astype(str).str.replace('.0', '', regex = False)
wms_telkom['SKU WH'] = wms_telkom['SKU WH'].astype(str)

cek=temp_telkom.copy()
temp_telkom = temp_telkom.merge(wms_telkom.drop_duplicates(['Order #', 'SKU WH']), how = 'left', left_on = ['Order #', 'Real SKU'], right_on = ['Order #', 'SKU WH'])

# temp_telkom['Warehouse Quantity'] = temp_telkom['Warehouse Quantity'] * temp_telkom['Qty. Invoiced']/temp_telkom.groupby(['Order #', 'Real Nama Produk'])['Qty. Invoiced'].transform('sum')

temp_telkom_bundle = temp_telkom[temp_telkom['Brand'] == 'Bundle']

temp_buat_bundle = temp_telkom[temp_telkom['Warehouse Quantity'].notnull()].groupby(['Order #', 'Bundle Name']).agg({'WH' : 'first', 'SO STATUS' : 'first', 'Warehouse Quantity' : 'max', 'Qty. Invoiced' : 'max'}).reset_index()
temp_buat_bundle['Warehouse Quantity'] = temp_buat_bundle['Warehouse Quantity']/temp_buat_bundle['Qty. Invoiced']
temp_buat_bundle = temp_buat_bundle.drop('Qty. Invoiced', axis = 1)

temp_telkom_bundle = temp_telkom_bundle.merge(temp_buat_bundle.drop_duplicates(['Order #', 'Bundle Name']), how = 'left', left_on = ['Order #', 'Product Name'], right_on = ['Order #', 'Bundle Name']).set_index(temp_telkom_bundle.index)
temp_telkom_bundle['Warehouse Quantity_y'] = temp_telkom_bundle['Warehouse Quantity_y']*temp_telkom_bundle['Qty. Invoiced']

temp_telkom['SO STATUS'][temp_telkom_bundle.index] = temp_telkom_bundle['SO STATUS_y']
temp_telkom['WH'][temp_telkom_bundle.index] = temp_telkom_bundle['WH_y']
temp_telkom['Warehouse Quantity'][temp_telkom_bundle.index] = temp_telkom_bundle['Warehouse Quantity_y']

for_wms = for_wms[~for_wms['Order #'].isin(wms_telkom['Order #'])]
for_wms = for_wms.append(temp_telkom, ignore_index = True, sort = False)

done masking


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [15]:
## just for display
pd.options.display.max_columns = 150

In [19]:
# for_wms.to_csv(r'D:\Backup\Backup 26 Mei\Settlement Project Backup/Tiktok Qty.csv', index = False, sep = ';')

In [17]:
# for_wms[
#     (for_wms['Order #'].isin(wms_telkom[wms_telkom['SO STATUS'] != 'cancelled']['Order #'])) &
#     (for_wms['Warehouse Quantity'].isnull())
# ].groupby(['Real SKU', 'Product Name'])['Order #'].first().reset_index()

In [18]:
# display(for_wms[for_wms['Order #']=='576597372653373571'])
# wms_telkom[wms_telkom['Order #']=='576597372653373571']

In [ ]:
### old template

import pandas as pd
import numpy as np
import os

set_tiktok = pd.DataFrame()

cols = pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/Tiktok.xlsx').columns.to_list()

for i in os.listdir(r'Settlement Project/Manual/Tiktok/Settlement/'):
    if r'$' not in i:
        temp = pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/' + str(i))
        temp['ID'] = temp['ID'].astype(str).str.replace('\t', '', regex = False)
        temp['Source'] = i
        print(i)
        if '2022' in i:
            cols = cols[:4] + ['Total Earnings From Order'] + cols[4:5] + ['Product Price', 'Customer Shipping Fee', ] + cols[5:6] + ['Total Fees'] + cols[6:7] + ['Customer Order Refund', 'Platform Discount Refund'] + cols[7:9] + ['Actual Shipping Fee', 'Platform Shipping Fee Discount'] + cols[9:-1] + ['Related Order ID']  
            cols = cols  + ['Source']
            temp.columns = cols ###komen31 may
            print(cols)
            
            
        true_col = ['no', 'ID', 'Type', 'Time', 'Total Earnings From Order', 'User Pay', 'Product Price', 'Customer Shipping Fee', 'Platform Promotion', 'Total Fees', 'Refund', 'Customer Order Refund', 'Platform Discount Refund', 'Payment Fee', 'Shipping Fee', 'Actual Shipping Fee', 'Platform Shipping Fee Discount', 'Affiliate Commission', 'Shipping Fee Adjustment', 'Shipping Fee Rebate(IDR)', 'Charge Back', 'Customer Service Compensation', 'Promotion Adjustment', 'Other Adjustment', 'Settlement Amount', 'Related Order ID', 'Source']
        set_tiktok = set_tiktok.append(temp, ignore_index = True, sort = False)
        if temp.columns.to_list() == true_col:
            print('Column Sama')
        else :
            print('Column Beda')
            print([x for x in true_col if x not in temp.columns.to_list()])
            print([x for x in temp.columns.to_list() if x not in true_col])

In [19]:
### run -10
### new template

import pandas as pd
import numpy as np
import os

set_tiktok = pd.DataFrame()
cols = pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/Tiktok.xlsx').columns.to_list()
dropcol=pd.read_excel(r"Settlement Project\Manual\Tiktok\Settlement\New folder\ren_col_juni22.xlsx",sheet_name='Sheet1')
rencol=pd.read_excel(r"Settlement Project\Manual\Tiktok\Settlement\New folder\ren_col_juni22.xlsx",sheet_name='Sheet2')
dropcol2=pd.read_excel(r"Settlement Project\Manual\Tiktok\Settlement\New folder\ren_col_juni22.xlsx",sheet_name='Sheet4')
rencol2=pd.read_excel(r"Settlement Project\Manual\Tiktok\Settlement\New folder\ren_col_juni22.xlsx",sheet_name='Sheet3')

cols = pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/Tiktok.xlsx').columns.to_list()


for file in os.listdir(r'Settlement Project/Manual/Tiktok/Settlement/'):
    listsheet=[]
    if '.xlsx' in file:
        print(file)
        temp = pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/' + str(file),sheet_name=None)
        for name,sheet in temp.items():
            if 'Settlements' in name:
                listsheet.append(name)
        # print(listsheet)
        if len(listsheet)>1:
            for i in listsheet:
                print(i)
                temp=pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/' + str(file),sheet_name=i)
                if 'before 2022.6.6' in i:
                    temp=temp.drop(columns=dropcol['drop'].unique()).rename(columns=dict(rencol.values))
                    temp['ID'] = temp['ID'].astype(str).str.replace('\t', '', regex = False)
                    temp['Source'] = file
                    temp=temp[[x for x in temp.columns.to_list() if 'Unnamed' not in x]]
                    # print(i)
                    if '2022' in i:
                        cols = cols[:4] + ['Total Earnings From Order'] + cols[4:5] + ['Product Price', 'Customer Shipping Fee', ] + cols[5:6] + ['Total Fees'] + cols[6:7] + ['Customer Order Refund', 'Platform Discount Refund'] + cols[7:9] + ['Actual Shipping Fee', 'Platform Shipping Fee Discount'] + cols[9:-1] + ['Related Order ID']  
                        cols = cols  + ['Source']
                        temp.columns = cols 
                        # display(temp[temp['ID']=='576536911503788463'])
                        
                else:
                    # print('masuk elif')
                    temp=temp.drop(columns=dropcol2['drop'].unique()).rename(columns=dict(rencol2.values))
                    temp['ID'] = temp['ID'].astype(str).str.replace('\t', '', regex = False)
                    temp['Source'] = file
                    temp=temp[[x for x in temp.columns.to_list() if 'Unnamed' not in x]]
                if 'ID' in set_tiktok.columns:
                    temp=temp[~temp['ID'].isin(set_tiktok['ID'].unique())]
                set_tiktok = set_tiktok.append(temp, ignore_index = True, sort = False)
                # display(set_tiktok[set_tiktok['ID']=='576536911503788463'])
            listsheet=[]
        
            
        else:
            
            # print(file)
            # temp = pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/' + str(i))
            temp = pd.read_excel(r'Settlement Project/Manual/Tiktok/Settlement/' + str(file))
            if len(listsheet)>0 and listsheet[0]=='Settlements after 2022.6.6':
                print('2023')
                dropcol3=dropcol2[dropcol2['drop'].isin(temp.columns)]
                temp=temp.drop(columns=dropcol3['drop'].unique()).rename(columns=dict(rencol2.values))
            temp['ID'] = temp['ID'].astype(str).str.replace('\t', '', regex = False)
            temp['Source'] = file
            # print(i)
            if '2022' in file:
                cols = cols[:4] + ['Total Earnings From Order'] + cols[4:5] + ['Product Price', 'Customer Shipping Fee', ] + cols[5:6] + ['Total Fees'] + cols[6:7] + ['Customer Order Refund', 'Platform Discount Refund'] + cols[7:9] + ['Actual Shipping Fee', 'Platform Shipping Fee Discount'] + cols[9:-1] + ['Related Order ID']  
                cols = cols  + ['Source']
                temp.columns = cols 
                # print(cols)
            if 'ID' in set_tiktok.columns:
                temp=temp[~temp['ID'].isin(set_tiktok['ID'].unique())]
            set_tiktok = set_tiktok.append(temp, ignore_index = True, sort = False)
set_tiktok=set_tiktok[set_tiktok['Type']!='Type']
set_tiktok_raw=set_tiktok.copy()

06. Settlement Tiktok 2023.xlsx
2023
Settlement TikTok 2022 - TERUPDATE.xlsx
Settlements before 2022.6.6
Settlements after 2022.6.6
Tiktok.xlsx


In [20]:
### for backup
set_tiktok=set_tiktok_raw.copy()

In [21]:
### run -11
### harus kosong
display(set_tiktok[set_tiktok.duplicated()])
cek=set_tiktok.groupby('ID')[['Source']].nunique().reset_index()
cek[cek['Source']>1]


,no,ID,Type,Time,User Pay,Subtotal before discounts,Platform Promotion,Refund(IDR),Refund of seller discounts,Payment Fee,...,Affiliate Commission,Shipping Fee Adjustment,Shipping Fee Rebate(IDR),Charge Back,Customer Service Compensation,Promotion Adjustment,Other Adjustment,Settlement Amount,Related Order ID,Currency


,ID,Source


In [22]:
### run -12
set_tiktok = set_tiktok[set_tiktok['no'] != 'no']
set_tiktok['ID'] = set_tiktok['ID'].astype(str).str.replace('\t', '', regex = False)
set_tiktok['Shipping Fee'] = pd.to_numeric(set_tiktok['Customer Shipping Fee'], errors = 'coerce').fillna(0) - pd.to_numeric(set_tiktok['Shipping Fee'], errors = 'coerce').fillna(0)
set_tiktok['Refund'] = pd.to_numeric(set_tiktok['Refund'], errors = 'coerce').fillna(0) ### timo 1 juli 2022


### timo jan 2023
set_tiktok.loc[set_tiktok['Settlement Amount'].isnull(),'Settlement Amount']=set_tiktok['Settlement amount']
set_tiktok.loc[set_tiktok['Affiliate Commission'].isnull(),'Affiliate Commission']=set_tiktok['Affiliate commission']




# set_tiktok = set_tiktok[['ID', 'Refund', 'Payment Fee', 'Settlement Amount', 'Shipping Fee']]## komen 23 sept
set_tiktok = set_tiktok[['ID', 'Refund', 'Payment Fee', 'Settlement Amount', 'Shipping Fee','Affiliate Commission']]
# set_tiktok['Payment Fee']=abs(set_tiktok['Payment Fee'].astype(int)) ### 20 juli ## komen 23 sept
set_tiktok['Payment Fee']=abs(set_tiktok['Payment Fee'].astype(int)+set_tiktok['Affiliate Commission'].astype(int)) ### 23 sept
set_tiktok = set_tiktok.rename(columns = {'Payment Fee' : 'Comfee'}) #### 1 juli 2022 timo

# set_tiktok['Settlement Amount'] = pd.to_numeric(set_tiktok['Settlement Amount'], errors = 'coerce').fillna(0) + pd.to_numeric(set_tiktok['Payment Fee'], errors = 'coerce').fillna(0)

for_wms['Order #'] = for_wms['Order #'].astype(str)
for_wms['Total Net Before PPN'] = for_wms['Total Net']/1.11
# paid_tiktok = for_wms.merge(set_tiktok, how = 'left', left_on = 'Order #', right_on = 'ID')

In [73]:
set_tiktok

,ID,Refund,Comfee,Settlement Amount,Shipping Fee,Affiliate Commission
0,576898070669003058,0.0,4877,282823,0.0,0
1,576900879547074779,0.0,2884,85516,0.0,0
2,576900879404141463,0.0,2884,85516,0.0,0
3,576900940193237515,0.0,3060,102940,0.0,0
4,576901679735670954,0.0,3420,138580,0.0,0
...,...,...,...,...,...,...
62027,576463937633814157,0.0,147,20203,-4838.0,0
62028,576463939166504823,0.0,270,36686,-1044.0,0
62029,576463939133933444,0.0,147,20203,-4838.0,0
62030,576463976343569677,0.0,147,20203,-4838.0,0


In [23]:
## run -13
bundling = for_wms[for_wms['Bundle Name'].notnull()]
real_trx = for_wms[for_wms['Bundle Name'].isnull()]

real_trx['Order #'] = real_trx['Order #'].astype(str)
set_tiktok['ID'] = set_tiktok['ID'].astype(str)

real_trx = real_trx.merge(set_tiktok, how = 'left', left_on = 'Order #', right_on = 'ID')

non_prop = real_trx[real_trx['Brand'].isin(['Bonus Produk', 'Gimmick'])]
real_trx = real_trx[~real_trx['Brand'].isin(['Bonus Produk', 'Gimmick'])]

real_trx['Settlement Amount'] = pd.to_numeric(real_trx['Settlement Amount'], errors = 'coerce')
# real_trx['Payment Fee'] = pd.to_numeric(real_trx['Payment Fee'], errors = 'coerce') #komen 1 juli 22
real_trx['Comfee'] = pd.to_numeric(real_trx['Comfee'], errors = 'coerce') # baru 1 juli

# real_trx['Total Produk'] = real_trx['Settlement Amount'] + real_trx['Payment Fee'].fillna(0) #komen 1 juli 22
real_trx['Total Produk'] = real_trx['Settlement Amount'] + real_trx['Comfee'].fillna(0) # baru 1 juli


real_trx['Total Produk Prop'] = real_trx['Total Produk'].fillna(0) * real_trx['Total Net Before PPN']/real_trx.groupby(['Order #'])['Total Net Before PPN'].transform('sum')
# real_trx['Payment Fee Prop'] = real_trx['Payment Fee'].fillna(0) * real_trx['Qty. Invoiced']/real_trx.groupby(['Order #'])['Qty. Invoiced'].transform('sum') #komen 1juli 2022
real_trx['Comfee Prop'] = real_trx['Comfee'].fillna(0) * real_trx['Qty. Invoiced']/real_trx.groupby(['Order #'])['Qty. Invoiced'].transform('sum') # baru 1 juli 2022
real_trx['Shipping Fee Prop'] = real_trx['Shipping Fee'].fillna(0) * real_trx['Total Net Before PPN']/real_trx.groupby(['Order #'])['Total Net Before PPN'].transform('sum')

non_prop['Total Produk Prop'] = 0
# non_prop['Payment Fee Prop'] = 0 # komen 1 juli 2022
non_prop['Comfee Prop'] = 0 #baru 1 juli 2022
non_prop['Shipping Fee Prop'] = 0

real_trx = real_trx.append(non_prop, ignore_index = True, sort = False)

bundling['Order #'] = bundling['Order #'].astype(str)

# bundling = bundling.merge(real_trx[['Order #', 'Product Name', 'Total Produk Prop', 'Payment Fee Prop', 'Shipping Fee Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])
bundling = bundling.merge(real_trx[['Order #', 'Product Name', 'Total Produk Prop', 'Comfee Prop', 'Shipping Fee Prop']].drop_duplicates(['Order #', 'Product Name']), how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name'])

non_prop = bundling[bundling['Brand'].isin(['Bonus Produk', 'Gimmick'])]
bundling = bundling[~bundling['Brand'].isin(['Bonus Produk', 'Gimmick'])]

bundling['Total Produk Prop'] = bundling['Total Produk Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
# bundling['Payment Fee Prop'] = bundling['Payment Fee Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundling['Comfee Prop'] = bundling['Comfee Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')                                  
bundling['Shipping Fee Prop'] = bundling['Shipping Fee Prop'].fillna(0) * bundling['Total Net Before PPN']/bundling.groupby(['Order #', 'Bundle Name'])['Total Net Before PPN'].transform('sum')


non_prop['Total Produk Prop'] = 0
# non_prop['Payment Fee Prop'] = 0
non_prop['Comfee Prop'] = 0                                    
non_prop['Shipping Fee Prop'] = 0

bundling = bundling.append(non_prop, ignore_index = True, sort = False)

# real_trx = real_trx.drop(['SKU_y', 'Real SKU_y', 'SKU_y'], axis = 1)
# real_trx = real_trx.rename(columns = {'SKU_x' : 'SKU'})

# bundling = bundling.drop(['SKU_y', 'Real SKU_y', 'SKU_y', 'Product Name_y'], axis = 1)
# bundling = bundling.rename(columns = {'SKU_x' : 'SKU', 'Product Name_x' : 'Product Name'})

total_trx = real_trx.append(bundling, ignore_index = True, sort = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
### run -14
total_trx['Paid'] = np.nan

indeks = total_trx[total_trx['Order #'].isin(set_tiktok['ID'])].index.to_list()
total_trx['Paid'][indeks] = 'Paid'

set_tiktok['Settlement Amount'] = pd.to_numeric(set_tiktok['Settlement Amount'], errors = 'coerce').fillna(0)
# set_tiktok['Payment Fee'] = pd.to_numeric(set_tiktok['Payment Fee'], errors = 'coerce').fillna(0)
set_tiktok['Comfee'] = pd.to_numeric(set_tiktok['Comfee'], errors = 'coerce').fillna(0)

# indeks = total_trx[total_trx['Order #'].isin(set_tiktok[
#     (set_tiktok['Refund'].notnull()) &
#     (set_tiktok['Refund'] != 0) &
#     (set_tiktok['Settlement Amount'] + set_tiktok['Payment Fee'] <= 0)
# ]['ID'])].index.to_list()

indeks = total_trx[total_trx['Order #'].isin(set_tiktok[
    (set_tiktok['Refund'].notnull()) &
    (set_tiktok['Refund'] != 0) &
    (set_tiktok['Settlement Amount'] + set_tiktok['Comfee'] <= 0)
]['ID'])].index.to_list()
total_trx['Paid'][indeks] = 'Refund'

# indeks = total_trx[total_trx['Order #'].isin(set_tiktok[
#     (set_tiktok['Refund'].notnull()) &
#     (set_tiktok['Refund'] != 0) &
#     (set_tiktok['Settlement Amount'] + set_tiktok['Payment Fee'] > 0)
# ]['ID'])].index.to_list()
indeks = total_trx[total_trx['Order #'].isin(set_tiktok[
    (set_tiktok['Refund'].notnull()) &
    (set_tiktok['Refund'] != 0) &
    (set_tiktok['Settlement Amount'] + set_tiktok['Comfee'] > 0)
]['ID'])].index.to_list()
total_trx['Paid'][indeks] = 'Half Paid'

total_trx['Paid'] = total_trx['Paid'].fillna('Unpaid')

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [25]:
### run -15
total_trx['Total Produk Prop Adj'] = total_trx['Total Produk Prop']/1.11
total_trx['BTL Cost'] = (total_trx['Total Produk Prop Adj']) - total_trx['Total Net Before PPN']
# total_trx['Payment Fee Prop'] = total_trx['Payment Fee Prop'] * (-1)
total_trx['Comfee Prop'] = total_trx['Comfee Prop'] * (-1)

In [26]:
### run -16
total_trx['Product Name'] = total_trx['Product Name'].fillna(total_trx['Product Name_x'])

total_trx = total_trx.drop(['Product Name_x', 'Product Name_y'], axis = 1)

In [27]:
### run -17

# total_trx_full = total_trx[['Order #', 'Order Status', 'SKU', 'Product Name', 'Qty. Invoiced',
#        'Order date', 'Paid Date', 'AWB', 'Shipping Courier', 'Region Group',
#        'Sales Order ID', 'Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand',
#          'Bundle Name',
#        'Price List NFI', 'Total Net', 'Bundle Flag', 'Parent Item',
#        'Parent SKU', 'Harga Cost', 'Total Harga Cost', 'SKU WH', 'SO STATUS',
#        'Warehouse Quantity', 'WH', 'Total Net Before PPN', 'ID', 'Refund',
#        'Payment Fee', 'Shipping Fee', 'Settlement Amount', 'Total Produk',
#        'Total Produk Prop', 'Payment Fee Prop', 'Shipping Fee Prop', 'Paid',
#        'Total Produk Prop Adj', 'BTL Cost']]
total_trx_full = total_trx[['Order #', 'Order Status', 'SKU', 'Product Name', 'Qty. Invoiced',
       'Order date', 'Paid Date', 'AWB', 'Shipping Courier', 'Region Group',
       'Sales Order ID', 'Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand',
         'Bundle Name',
       'Price List NFI', 'Total Net', 'Bundle Flag', 'Parent Item',
       'Parent SKU', 'Harga Cost', 'Total Harga Cost', 'SKU WH', 'SO STATUS',
       'Warehouse Quantity', 'WH', 'Total Net Before PPN', 'ID', 'Refund',
       'Comfee', 'Shipping Fee', 'Settlement Amount', 'Total Produk',
       'Total Produk Prop', 'Comfee Prop', 'Shipping Fee Prop', 'Paid',
       'Total Produk Prop Adj', 'BTL Cost','Pemilik Budget']]

total_trx_full['Year'] = pd.to_datetime(total_trx_full['Order date']).dt.year
total_trx_full['Month'] = pd.to_datetime(total_trx_full['Order date']).dt.month_name()
total_trx_full['Date'] = pd.to_datetime(total_trx_full['Order date']).dt.date
total_trx_full['Date'] = pd.to_datetime(total_trx_full['Order date']).dt.day # timo nov 2022


## tambahan timo Nov 2022
total_trx_full.loc[total_trx_full['Order date'].str.contains('/'),'Year']=pd.to_datetime(total_trx_full[total_trx_full['Order date'].str.contains('/')]['Order date'],format="%d/%m/%Y %H:%M:%S").dt.year
total_trx_full.loc[total_trx_full['Order date'].str.contains('/'),'Month']=pd.to_datetime(total_trx_full[total_trx_full['Order date'].str.contains('/')]['Order date'],format="%d/%m/%Y %H:%M:%S").dt.month_name()
total_trx_full.loc[total_trx_full['Order date'].str.contains('/'),'Date']=pd.to_datetime(total_trx_full[total_trx_full['Order date'].str.contains('/')]['Order date'],format="%d/%m/%Y %H:%M:%S").dt.day
# total_trx_full.loc[total_trx_full['Order date'].str.contains('/'),'Year']='year'
# total_trx_full.loc[total_trx_full['Order date'].str.contains('/'),'Month']='month'
# total_trx_full.loc[total_trx_full['Order date'].str.contains('/'),'Date']='date'

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [86]:
# total_trx_full[total_trx_full['Order #']=='576666467657878146']
# set_tiktok_raw[set_tiktok_raw['ID']=='576471963508901219']
# total_trx_full[total_trx_full.duplicated()]
# tiktok[tiktok['Order #']=='576666748996454552']

In [28]:
### run -18

import pandas as pd

prev = pd.read_csv(r"D:\6. settlement\Code\Settlement Project\Backup\Realisasi Apr 2023\Final Result\Raw Unpivoted\CSV\Running TikTok Apr 23.csv", sep = ';')
prev_month = 'April'
prev = prev.rename(columns = {'Paid' : 'Paid ' + prev_month})
current = total_trx_full.copy()

# current = pd.read_csv("D:\Backup\Backup 26 Mei\Settlement Project Backup\Realisasi Januari 2022\Final Result\Running Nutrimart Jan Jan 2022.csv", sep = ';')

prev['Order #'] = prev['Order #'].astype(str)
prev['Order #']=prev['Order #'].str.extract('(\d+)')### tambahan timo nov 2022
prev['Real SKU'] = prev['Real SKU'].astype(str)

current['Order #'] = current['Order #'].astype(str)
current['Real SKU'] = current['Real SKU'].astype(str)

merged = current.merge(prev[['Order #','Real SKU', 'Paid ' + prev_month, 'Bundle Name']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', on = ['Order #', 'Real SKU', 'Bundle Name'])
merged_null = merged[merged['Paid ' + prev_month].isnull()]
merged_null = merged_null.drop('Paid ' + prev_month, axis = 1)
merged_null = merged_null.merge(prev[['AWB','Real SKU', 'Paid ' + prev_month, 'Bundle Name']].drop_duplicates(['AWB', 'Real SKU', 'Bundle Name']), how = 'left', on = ['AWB', 'Real SKU', 'Bundle Name']).set_index(merged_null.index)
merged['Paid ' + prev_month][merged_null.index] = merged_null['Paid ' + prev_month]
# display(merged_null[merged_null['Order #']=='576462415439627454'])
# display(merged[merged['Order #']=='576462415439627454'])

import numpy as np

merged['Paid Beda'] = np.nan

indeks = merged[merged['Paid'] != merged['Paid ' + prev_month]].index.to_list()
merged['Paid Beda'][indeks] = 'Beda'

merged['Paid Beda'] = merged['Paid Beda'].fillna('Sama')
# merged[merged['Paid Beda'] == 'Beda'][['Order #','Brand','Real SKU','Month', 'Paid_x', 'Paid_y', 'Paid Beda']]
merged[merged['Paid Beda'] == 'Beda'].groupby(['Year', 'Month', 'Paid', 'Paid ' + prev_month], dropna = False)['Order #'].nunique()


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (15,19,23,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Year  Month  Paid    Paid April
2023  April  Paid    Unpaid         923
                     NaN           1646
             Unpaid  NaN            236
      March  Paid    Unpaid           4
      May    Paid    NaN           7650
             Unpaid  NaN           3377
Name: Order #, dtype: int64

In [29]:
# run -19
merged['Order #'] = '="' + merged['Order #'] + '"'
merged['Sales Order ID'] = '="' + merged['Sales Order ID'] + '"'

In [30]:
# run -20
merged = merged.rename(columns = {'Paid' : 'Paid May'})


In [31]:
# run -21
merged.to_csv('Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\CSV\Running TikTok May 23.csv', sep = ';', index = False)

In [32]:
# run -22
merged.to_excel('Settlement Project\Backup\Realisasi May 2023\Final Result\Raw Unpivoted\Excel\Running TikTok May 23.xlsx', index = False)

In [ ]:
#### done tiktok yayyy #####

In [2]:
import requests
order_id = "WA-1646041245"
url = "https://app.yellow.ai/api/data/data/search/0-1?bot=x1631078626373&field=channel_order_id&query=" + order_id + "&collection=transaction_order"


payload = {}
headers = {
    'x-auth-token': 'e08ca85366b275af5788f621183c5bbbc31d4d0318a9fd0ecda068f880440f8b'
}

response = requests.request("GET", url, headers=headers, data=payload)

In [3]:
response

<Response [200]>

In [71]:


if len([x for x in response.json()['data'] if x['channel_order_id'] == order_id]) != 0:
    ref_id = [x for x in response.json()['data'] if x['channel_order_id'] == order_id][0]['_id']
    shipping_phone = [x for x in response.json()['data'] if x['channel_order_id'] == order_id][0]['shipping_phone']
    shipping_phone = re.sub('^0', '62', str(shipping_phone))

In [73]:
[x for x in response.json()['data'] if x['channel_order_id'] == order_id]

[]

In [10]:
[x for x in response.json()['data']]

['records', 'total']

In [19]:
len(response.json()['data']['records'][0])

29

In [56]:
data = pd.read_excel(r"D:\Backup\Backup 26 Mei\Settlement Project Backup\Output File\Tiktok\Running Tiktok Jan 21 - Feb 22.xlsx", sheet_name = 'Running Tiktok Jan 21 - Feb 22')

In [57]:
data.to_csv('D:\Backup\Backup 26 Mei\Settlement Project Backup\Running Tiktok Jan 21 - Feb 22 Testing.csv', sep = ';', index = False, float_format='%g')

In [49]:
data['Order #'] = data['Order #'].astype(str)

In [51]:
data

,Order #,Order Status,SKU,Product Name,Qty. Invoiced,Order date,Paid Date,AWB,Shipping Courier,Region Group,Sales Order ID,Real SKU,Real Nama Produk,Brand,Sub Brand,Bundle Name,Price List NFI,Total Net,Bundle Flag,Parent Item,Parent SKU,Harga Cost,Total Harga Cost,SKU WH,SO STATUS,Warehouse Quantity,WH,Total Net Before PPN,ID,Refund,Payment Fee,Comfee,Settlement Amount,Total Produk,Total Produk Prop,Payment Fee Prop,Comfee Prop,Paid February,Total Produk Prop Adj,BTL Cost,Year,Month,Date,Paid January,Paid Beda
0,576464768098468992,Canceled,2104508105,Tropicana Slim Korean Garlic Butter Cookies (5...,1,2021-11-30 15:52:45,NaT,NaN,NaN,Banten,576464768098468992,2104508105,Tropicana Slim Korean Garlic Butter Cookies 5 Sch,TS,Tropicana Slim Kuning,NaN,21450,21450,NaN,NaN,NaN,21450,21450,NaN,NaN,NaN,NaN,19500.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,Unpaid,0.000000,-19500.000000,2021,November,2021-11-30,Unpaid,Sama
1,576464646991480064,Canceled,1101558453,NutriSari Jeruk Nipis (40 Sch),1,2021-11-29 13:48:46,NaT,NaN,NaN,Jawa Tengah,576464646991480064,1101558453,Nutrisari Jeruk Nipis 40'sachet (4 Renceng),NS,NutriSari,NaN,42900,42900,NaN,NaN,NaN,42900,42900,NaN,NaN,NaN,NaN,39000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,Unpaid,0.000000,-39000.000000,2021,November,2021-11-29,Unpaid,Sama
2,576464609319159040,Canceled,2104523105,Tropicana Slim Hokkaido Cheese Cookies (5 Sch)...,2,2021-11-29 01:45:25,NaT,NaN,NaN,Banten,576464609319159040,2104523105,Tropicana Slim Hokkaido Cheese Cookies,TS,Tropicana Slim Kuning,NaN,21450,42900,NaN,NaN,NaN,21450,42900,NaN,NaN,NaN,NaN,39000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,Unpaid,0.000000,-39000.000000,2021,November,2021-11-29,Unpaid,Sama
3,576464609093584000,Canceled,2104523105,Tropicana Slim Hokkaido Cheese Cookies (5 Sch)...,2,2021-11-29 01:24:26,NaT,NaN,NaN,Banten,576464609093584000,2104523105,Tropicana Slim Hokkaido Cheese Cookies,TS,Tropicana Slim Kuning,NaN,21450,42900,NaN,NaN,NaN,21450,42900,NaN,NaN,NaN,NaN,39000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,Unpaid,0.000000,-39000.000000,2021,November,2021-11-29,Unpaid,Sama
4,576464540403467008,Canceled,2104543105,Tropicana Slim Korean Goguma Cookies (5 Sch) -...,1,2021-11-28 11:53:10,NaT,NaN,NaN,Daerah Istimewa Yogyakarta,576464540403467008,2104543105,Tropicana Slim Korean Goguma Cookies (5 Sch),TS,Tropicana Slim Kuning,NaN,21450,21450,NaN,NaN,NaN,21450,21450,NaN,NaN,NaN,NaN,19500.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0,Unpaid,0.000000,-19500.000000,2021,November,2021-11-28,Unpaid,Sama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8179,576485497347015936,Completed,2101461,HiLo Teen RTD Coklat 200ml,4,2022-02-08 16:58:29,2022-02-08 17:27:12,JX0037337385,J&T Express,Jawa Barat,576485497347015936,2101461,HiLo Teen RTD Coklat 200ml,HiLo,HiLo Teen,HiLo Teen Chocolate 200ml (4 Pcs) - Ready to D...,6600,26400,NaN,HiLo Teen RTD Chocolate 200ml,NaN,6600,26400,2.101461e+06,confirmed,4.0,Telkom,24000.0,NaN,NaN,NaN,NaN,NaN,NaN,24867.448680,245.000000,0,Paid,22606.771528,-1393.228472,2022,February,2022-02-08,NaN,Beda
8180,576484626195056000,Completed,2104153110,Tropicana Slim Cafe Latte (10 sch),2,2022-02-06 23:06:50,2022-02-06 23:07:29,JX0035606877,J&T Express,Jakarta,576484626195056000,2104153110,Tropicana Slim Cafe Latte (10 sch),TS,Tropicana Slim Kuning,Tropicana Slim Cafe Latte [10 sachets x 2 pcs],29150,58300,NaN,Tropicana Slim Cafe Latte (10 sch),2.104153e+09,29150,58300,2.104153e+09,confirmed,2.0,Telkom,53000.0,NaN,NaN,NaN,NaN,NaN,NaN,38609.322222,221.333333,0,Paid,35099.383838,-17900.616162,2022,February,2022-02-06,NaN,Beda
8181,576483686799869056,Completed,2104153110,Tropicana Slim Cafe Latte (10 sch),2,2022-02-05 08:16:08,2022-02-05 08:16:17,JX0038794263,J&T Express,West Java Province,576483686799869056,2104153110,Tropicana Slim Cafe Latte (10 sch),TS,Tropicana Slim Kuning,Tropicana Slim